**Air Quality Prediction and Classification:** <br> `Brief Info of the project:` <br>
The aim of this project is to develop a robust and intelligent system for air quality prediction and classification that integrates data from diverse sources, such as sensors and OpenWeather API. By employing advanced preprocessing techniques, feature alignment, and reinforcement learning, the system ensures compatibility and adaptability to dynamic data variations. The project further incorporates hybrid machine learning models for accurate prediction of air quality indices and categorization into health-impactful classifications. Ultimately, the solution aims to provide real-time insights to policymakers and general users, promoting informed decision-making and fostering healthier communities. <br>
`Dataset Info:` <br>
The data is collected realtime from OpenWhether using API. This dataset contains 13 features and most of them are numerical type. you can have more info about the resource in the link: https://openweathermap.org. <br>
In addition, we have gathered local data from the National Environmental Protection Agency of Afghanistan in Kabul city. And total it has 385 records from 3 different years (2024,2023,2022).
**Deep Learning Pipeline:**<br>
* Preprocessing.
* Data Cleaning.
* Feature Engineering.
* Model Selection.
* Model Evaluation.
* Deployment.

In [ ]:
# necessory libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline

### Preprocessing:
We have collected three dataset one from local government and the rest two others are belongs to Openweather API. This section contains the following steps.
* Data concatination
* Moving `local_time` from the end of columns to before aqi feature
* Null values
* Duplicate values
* Check Uniqueness of Countries and States
* Checking statistical Analysis

In [ ]:
# Kabul local data
kabul = pd.read_csv("This was Afghanistan local data")
kabul

In [ ]:
# openweather dataset
# It has two dataset and first we need to combine and then start preprocessing steps
first_data = pd.read_csv("add the dataset")
second_data = pd.read_csv("add the dataset")

In [ ]:
first_data.head(5)

In [ ]:
# first data info
first_data.info()

In [ ]:
# second data head
second_data.head()

In [ ]:
# second data info
second_data.info()

In [ ]:
# Move 'local_time' column to before 'aqi'. we need the feature to be the same position for concatination
columns = second_data.columns.tolist()
columns.remove('local_time')
aqi_index = columns.index('aqi')
columns.insert(aqi_index, 'local_time')
df = second_data[columns]

# lets Check the new column order
df.head()

In [ ]:
# now our two dataset is ready to be combined into a single data
data = pd.concat([df, first_data], ignore_index=True)
data.reset_index(drop=True, inplace=True)
data

In [ ]:
# checking total Afghanistan records before concatination with local data
data[data['country'] == "Afghanistan"]

In [ ]:
# total countries records in the dataset
np.array(data['country'].value_counts())

In [ ]:
# Concatinating Kabul dataset with main data
data = pd.concat([data,kabul], ignore_index=True)
data

In [ ]:
# main data info
data.info()

In [ ]:
# Afghanistan records after concatination
data[data['country'] == 'Afghanistan']

In [ ]:
# ploting total countries records
fig = px.scatter(data['country'].value_counts())
fig.show()

In [ ]:
# total countries records in the dataset
np.array(data['country'].value_counts())

In [ ]:
# Checking for null values
data.isna().sum()

**Null values Result:**
Totally we have null values in three main features `(aqi, no, nh3)`.<br>
Let's find out null values in visualizaiton. <br>
We will handle this efficently in Data Cleaning section.

In [ ]:
# Bar plot of missing values for each column
drop = ['country', 'state', 'local_time','longitude', 'latitude']
missing_values = data.drop(drop, axis=1).isnull().sum()
missing_values = missing_values[missing_values > 0]
missing_values.sort_values(inplace=True)

plt.figure(figsize=(14, 10))
missing_values.plot(kind='barh', color='orange')
plt.title('Missing Values Count by Column', fontsize=12)
plt.xlabel('Number of Missing Values', fontsize=10)
plt.ylabel('Columns', fontsize=10)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Checking duplicate values
duplicates = data.duplicated()
print(f"Total duplicates: {duplicates.sum()}")

In [ ]:
# Checking inside dataframe which records have the most
data[duplicates]

In [ ]:
# checking duplicates without some features
drop = ['country', 'state','aqi','longitude', 'latitude']
duplicates = data.drop(drop, axis=1).duplicated()
print(f"Total duplicates: {duplicates.sum()}")

In [ ]:
# lets check duplicates for each features individualy
for feature in data.columns:
  dup = data[feature].duplicated().sum()
  print(f"Total {feature} duplicates: {dup}")

In [ ]:
# Check Unique values of countries
np.array(data['country'].unique())

In [ ]:
# ploting the countries using plotly express
fig = px.scatter(data['country'].unique())
fig.update_layout(
    xaxis_title='Number of Countries avaliable',
    yaxis_title='Some First Countries Name'
    )
fig.show()

In [ ]:
# Check Unique states
np.array(data['state'].unique())

In [ ]:
# Checking Statistical Analysis
data.describe()

In [ ]:
from scipy.stats import norm
# Checking the statistical anlysis using Normal distribution plot for every feature
# Visualizing Statistacal Analysis without object types and [latitude, longitude, aqi, ...] features
drop = ['country', 'state', 'local_time', 'aqi', 'latitude', 'longitude']

for feature in data.drop(drop,axis=1).columns:
    plt.figure(figsize=(8, 4))
    plt.hist(data[feature], bins=30, color='steelblue', edgecolor='black', density=True)
    plt.title(f'Histogram of {feature}', fontsize=14)
    plt.xlabel(feature, fontsize=12)
    plt.ylabel('Density', fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.show()

### Result:
This is the basic statistical analysis, because we have null values in our dataset and requires to fill them and we will handle it in `Data Cleaning Section`.

**Resource For Skewness**:<br>
[check here](https://www.quora.com/How-do-you-know-if-a-data-set-is-skewed)


### Data Cleaning Section
In this section we are going to clean the data and following bellow's steps:
* Fill Kabul local data missing values.
* Check the distribution and fix them.
* Checking Correlation and Covariance and other advance methods.
* Handling Kabul's date format.
* Data Transformation handling.
* Checking & Handling Outliers

In [ ]:
# As Afghanistan data has 2 main features less than openweather, and these two features are not so efficient in AQI. We will drop those in feature Engineering.
data.isna().sum()

In [ ]:
# Check the null values of NO as dataframe
data[data['no'].isna()]

In [ ]:
# Checking statistical of Afghanistan data only
data[data['state'] == 'Afghanistan'].describe()

In [ ]:
# Checking statistical of Kabul local data
data[data['state'] == 'Kabul'].describe()

### Few visualization:
Lets have a small visualization of Kabul local data and Openweather data features KDE and time series plots.

**Further Analysis:**<br>
Before imputing or droping our missing data we need to analyze our data more and check the `NH3` and `NO` in time series. Because our dataset is a time series data and we need to check them before filling those features, and there are many methods to handle it such as `Liner Interpolation`, `KNN Imputation`, `FTLRI`, `Forward & Backward`... but we need an efficient one.

In [ ]:
# Lets check KDE (Kernal Desnity Plot) these features
def kde_plot(dataset):
  # unwanted features
  drop = ['latitude', 'longitude', 'aqi', 'local_time', 'state', 'country']

  # Kabul local data & openweather Kabul data
  kabul_kde = dataset[dataset['state'] == 'Kabul']
  openweather_kde = dataset[dataset['state'] == 'Afghanistan']

  # droping unwanted features
  dataset = dataset.drop(drop, axis=1)

  for feature in dataset:
    plt.figure(figsize=(12, 6))

    # KDE for Kabul data
    sns.kdeplot(kabul_kde[feature].dropna(), label=f'Kabul {feature}', fill=True, color='blue')

    # KDE for OpenWeather data
    sns.kdeplot(openweather_kde[feature].dropna(), label=f'OpenWeather {feature}', fill=True, color='green')

    plt.title(f'KDE Plot for {feature}')
    plt.xlabel(feature)
    plt.ylabel('Density')
    plt.legend()
    plt.show()

kde_plot(data)

### Kernel Density Estimation
You can check the links bellow for further information about KDE & Correlation:<br>
`KDE`: [here](https://towardsdatascience.com/kernel-density-estimation-explained-step-by-step-7cc5b5bc4517).<br>
`Correlation`: [here](https://medium.com/@rajneeshjha9s/measures-of-correlation-d8cae057085a#:~:text=Correlation%20is%20used%20to%20describe,with%20relation%20to%20each%20other.).

In [ ]:
# checking the correlation and covariance
"""
  Reminder:
  This is just checking data gathered from Openweather Afghanistan, Kabul region
  and Kabul officially gathered data from Government. This is not comparison between
  all openweather data and Kabul local data.
"""
def corr_and_cov(dataset):
  drop = ['latitude', 'longitude', 'aqi', 'local_time', 'state', 'country']
  kabul_corr = data[data['state'] == 'Kabul'].drop(drop, axis=1).corr() # Kabul local data correlation
  kabul_cov = data[data['state'] == 'Kabul'].drop(drop, axis=1).cov() # Kabul local data covariance

  openweather_corr = data[data['state'] == 'Afghanistan'].drop(drop, axis=1).corr() # Openweather Kabul data correlation
  openweather_cov = data[data['state'] == 'Afghanistan'].drop(drop, axis=1).cov() # Openweather Kabul data correlation

  # Visualize correlation
  plt.figure(figsize=(16, 12))

  plt.subplot(2, 2, 1)
  sns.heatmap(kabul_corr, annot=True, cmap='coolwarm', cbar=True, square=True)
  plt.title('Kabul Local Data Correlation')

  plt.subplot(2, 2, 2)
  sns.heatmap(openweather_corr, annot=True, cmap='coolwarm', cbar=True, square=True)
  plt.title('OpenWeather Kabul Data Correlation')

  # Visualize covariance
  plt.subplot(2, 2, 3)
  sns.heatmap(kabul_cov, annot=True, cmap='viridis', cbar=True, square=True)
  plt.title('Kabul Local Data Covariance')

  plt.subplot(2, 2, 4)
  sns.heatmap(openweather_cov, annot=True, cmap='viridis', cbar=True, square=True)
  plt.title('OpenWeather Kabul Data Covariance')

  plt.tight_layout()
  plt.show()

corr_and_cov(data)

### Result:
As you can see the `NO` and `NH3` is so correlated to other factors and this showes that we are going to **drop** these features in Feature Engineering section. But we can check the Temporal trend and after that lets see what will happen.<br>
**Idea behind droping:**<br>
`(NO)` & `(NH3)` are highly correlated with other pollutants such as `(CO)`, `(NO2)`, and `(SO2)`. Multicollinearity occurs when two or more predictors in a model are highly correlated, meaning they provide redundant information. In this case, "no" and "nh3" are strongly related to other features that are already in the model, leading to redundancy.


In [ ]:
# Drop unwantted features
drop = ['latitude', 'longitude', 'country', 'state', 'aqi']
openweather_data = data.drop(drop, axis=1)

# Visualize nh3 over time
plt.figure(figsize=(12, 6))
plt.plot(openweather_data.index, openweather_data['nh3'], label='NH3', color='blue')
plt.plot(openweather_data.index, openweather_data['no'], label='NO', color='green')
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Temporal Trends of NH3 and NO in OpenWeather Kabul Data')
plt.legend()
plt.show()

In [ ]:
# Rolling mean for trend smoothing
openweather_data['nh3_smoothed'] = openweather_data['nh3'].rolling(window=100).mean()
openweather_data['no_smoothed'] = openweather_data['no'].rolling(window=100).mean()

# Plot smoothed trends
plt.figure(figsize=(12, 6))
plt.plot(openweather_data['nh3_smoothed'], label='Smoothed NH3', color='blue')
plt.plot(openweather_data['no_smoothed'], label='Smoothed NO', color='green')
plt.title('Mean Smoothed Trends of NH3 and NO')
plt.legend()
plt.show()

`Brief Info of Temporal Trend Analysis`:<br>
Temporal trends analysis involves examining and modeling how data points change over time, which is fundamental in time series analysis.`Resource` [here](https://falconediting.com/en/blog/time-series-analysis-understanding-temporal-trends-and-patterns)<br>
`Brief Info of Rolling Mean Analysis`:<br>
A rolling average, also known as a moving average, is a statistical method used to analyze data points by creating averages of different subsets of the complete dataset. This technique is commonly used in time series analysis to smooth out short-term fluctuations and highlight longer-term trends or cycles. `Resource`: [here](https://www.quora.com/What-is-rolling-average).<br>
**Result:**<br>
Based on Check up as we have done, now we can impute our missing data using `FTLRI imputation` methods. If it does not work we will drop these features<br>
`Brief Info of FTLRI`: FTLRI is an effective time series air quality data imputation model that not only considers correlation, both in terms of time and attributes of the data points, but also legitimately utilizes logistic regression to deal with such correlation. `Resource` [here](https://www.mdpi.com/2073-4433/13/7/1044#:~:text=FTLRI%20is%20an%20effective%20time,to%20deal%20with%20such%20correlation). <br>
The results show that FTLRI has a significant advantage over the compared imputation approaches, both in the particular short-term and long-term time series air quality data. Furthermore, FTLRI has good performance on datasets with a relatively high missing rate, since it only selects the data extremely related to the missing values instead of relying on all the other data like other methods. <br>
`Resource`: Research Paper by Mei Chen, ORCID,Hongyu Zhu,Yongxu Chen and Youshuai Wang. <br>
 Click here for check up: [here](https://www.mdpi.com/2073-4433/13/7/1044)

In [ ]:
from sklearn.linear_model import LinearRegression

"""
To fill the missing values using FTLRI method we are going to classify our tasks into substeps
in the following steps we first distinguash the data into two parts first the data gathered from openweather API
and select Afghanistan records and the second one is Kabul Local data (as it has null values of no and nh3)
the we are going to train our logistic model on openweather Kabul data and then fill the missing data of
Kabul local data.
"""
# using a copy of the data
temp = data.copy()

# Step 1: Separate the dataset into training and target subsets
afghanistan_data = temp[(temp['state'] == 'Afghanistan') & temp[['no', 'nh3']].notnull().all(axis=1)]
kabul_data = temp[(temp['state'] == 'Kabul') & temp[['no', 'nh3']].isnull().any(axis=1)]

# Features used for prediction (excluding 'no' and 'nh3' themselves to prevent leakage)
features = [col for col in temp.columns if col not in ['no', 'nh3', 'state','longitude', 'latitude', 'country', 'local_time', 'aqi']]

# Step 2: Train a Linear Regression model for 'no' and 'nh3' using Afghanistan data
models = {}
for target in ['no', 'nh3']:
    X_train = afghanistan_data[features].dropna()
    y_train = afghanistan_data.loc[X_train.index, target]
    model = LinearRegression()
    model.fit(X_train, y_train)
    models[target] = model

# Step 3: Predict missing values for 'no' and 'nh3' in Kabul data
for target in ['no', 'nh3']:
    X_test = kabul_data[features].fillna(kabul_data[features].mean())
    predictions = models[target].predict(X_test)
    data.loc[X_test.index, target] = predictions

data

In [ ]:
# checking the correlation and covariance
"""
  Reminder:
  This is just checking data gathered from Openweather Afghanistan, Kabul region
  and Kabul officially gathered data from Government. This is not comparison between
  all openweather data and Kabul local data.
"""
def corr_and_cov(dataset):
  drop = ['latitude', 'longitude', 'aqi', 'local_time', 'state', 'country']
  kabul_corr = data[data['state'] == 'Kabul'].drop(drop, axis=1).corr() # Kabul local data correlation
  kabul_cov = data[data['state'] == 'Kabul'].drop(drop, axis=1).cov() # Kabul local data covariance

  openweather_corr = data[data['state'] == 'Afghanistan'].drop(drop, axis=1).corr() # Openweather Kabul data correlation
  openweather_cov = data[data['state'] == 'Afghanistan'].drop(drop, axis=1).cov() # Openweather Kabul data correlation

  # Visualize correlation
  plt.figure(figsize=(16, 12))

  plt.subplot(2, 2, 1)
  sns.heatmap(kabul_corr, annot=True, cmap='coolwarm', cbar=True, square=True)
  plt.title('Kabul Local Data Correlation')

  plt.subplot(2, 2, 2)
  sns.heatmap(openweather_corr, annot=True, cmap='coolwarm', cbar=True, square=True)
  plt.title('OpenWeather Kabul Data Correlation')

  # Visualize covariance
  plt.subplot(2, 2, 3)
  sns.heatmap(kabul_cov, annot=True, cmap='viridis', cbar=True, square=True)
  plt.title('Kabul Local Data Covariance')

  plt.subplot(2, 2, 4)
  sns.heatmap(openweather_cov, annot=True, cmap='viridis', cbar=True, square=True)
  plt.title('OpenWeather Kabul Data Covariance')

  plt.tight_layout()
  plt.show()

corr_and_cov(data)

In [ ]:
# checking kabul aqi
data[data['state'] == 'Kabul']

### Next step:
In this step we are going to fill Afghanistan local data aqi with the formula using by India and USA.<br>
AQI computing Formula: <br>
$$
I = \frac{(I_{\text{high}} - I_{\text{low}})}{(C_{\text{high}} - C_{\text{low}})} \times (C - C_{\text{low}}) + I_{\text{low}}
$$


where:<br>
I = the Air Quality index,<br>
C = the pollutant concentration,<br>
Clow= the concentration breakpoint that is ≤C,<br>
Chigh= the concentration breakpoint that is ≥C,<br>
Ilow= the index breakpoint corresponding to Clow,<br>
Ihigh= the index breakpoint corresponding to Chigh.<br>

In [ ]:
# filling Afghanistan local aqi using USA formula
# Define breakpoints for pollutants
breakpoints = {
    "so2": [(0, 19), (20, 79), (80, 249), (250, 349), (350, float('inf'))],
    "no2": [(0, 39), (40, 69), (70, 149), (150, 199), (200, float('inf'))],
    "pm10": [(0, 19), (20, 49), (50, 99), (100, 199), (200, float('inf'))],
    "pm2_5": [(0, 9), (10, 24), (25, 49), (50, 74), (75, float('inf'))],
    "o3": [(0, 59), (60, 99), (100, 139), (140, 179), (180, float('inf'))],
    "co": [(0, 4399), (4400, 9399), (9400, 12399), (12400, 15399), (15400, float('inf'))]
}

# USA Index ranges for AQI (qualitative classification ranges)
aqi_indices = [(0, 50), (51, 100), (101, 150), (151, 200), (201, 300)]

# calculate AQI for a pollutant
def calculate_aqi(concentration, pollutant):
    for (C_lo, C_hi), (I_lo, I_hi) in zip(breakpoints[pollutant], aqi_indices):
        if C_lo <= concentration < C_hi:
            return ((I_hi - I_lo) / (C_hi - C_lo)) * (concentration - C_lo) + I_lo
    return None  # if data is out of bound

# Map AQI values to their qualitative classification (1, 2, 3, 4, 5)
def map_aqi_to_classification(aqi):
    if aqi < 51:
        return 1  # Good
    elif aqi < 101:
        return 2  # Fair
    elif aqi < 151:
        return 3  # Moderate
    elif aqi < 201:
        return 4  # Abnormal
    else:
        return 5  # Dangerous

# Fill missing AQI values based on the calculated AQI from pollutants
def fill_aqi(row):
    if pd.isnull(row["aqi"]):
        aqi_values = []
        for pollutant in breakpoints.keys():
            if not pd.isnull(row[pollutant]):
                aqi = calculate_aqi(row[pollutant], pollutant)
                if aqi is not None:
                    aqi_values.append(aqi)
        # If we have valid AQI values, choose the maximum
        if aqi_values:
            max_aqi = max(aqi_values)
            return map_aqi_to_classification(max_aqi)  # Map to classification index
        else:
            return np.nan  # Return NaN if no valid AQI values found
    return row["aqi"]


copy = data.copy()
copy["aqi"] = copy.apply(fill_aqi, axis=1)

In [ ]:
# checking does we have null values in aqi feature
copy['aqi'].isna().sum()

In [ ]:
# checking total null values
copy.isna().sum()

### Relationship Analysis:
In these few cells we are going to check:
* Correlation
* Covarience
* MI(	Mutual Information )

#### Note:
We are checking the relationship of features with target column using MI but take note that we first are not considering (Amonia) and (Nitrogen monoxide) for the first time but the second plot we will, and it will help us for furthor decision making. And we will handle this on Feature Engineering. But we want to familiarize ourself with data in depth.

In [ ]:
# correlation and covarience with no & nh3
def corr_and_cov(dataset, drop_columns):

    filtered_data = dataset.drop(columns=drop_columns, axis=1)

    # Compute correlation and covariance
    corr = filtered_data.corr()  # Correlation matrix
    cov = filtered_data.cov()   # Covariance matrix

    # Dynamically adjust figsize based on feature count
    num_features = len(filtered_data.columns)
    figsize = (num_features * 2, num_features * 0.8)  # Adjust scaling factors as needed

    # Create subplots
    plt.figure(figsize=figsize)

    # Correlation heatmap
    plt.subplot(1, 2, 1)
    sns.heatmap(corr, annot=True, cmap='coolwarm', cbar=True, annot_kws={"size": 8}, fmt=".2f")
    plt.title('Data Correlation', fontsize=14)

    # Covariance heatmap
    plt.subplot(1, 2, 2)
    sns.heatmap(cov, annot=True, cmap='viridis', cbar=True, annot_kws={"size": 8}, fmt=".2f")
    plt.title('Data Covariance', fontsize=14)

    # Adjust layout
    plt.tight_layout()
    plt.show()

# Drop irrelevant features
drop_columns = ['latitude', 'longitude', 'local_time', 'state', 'country']
corr_and_cov(copy, drop_columns)

In [ ]:
# correlation and covarience without no & nh3
drop_columns = ['latitude', 'longitude', 'local_time', 'state', 'country', 'no', 'nh3']
corr_and_cov(copy, drop_columns)

Lets go further analysis with MI or mutual information.

In [ ]:
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
# Mutual Information classification without (NH3, NO)

def calculate_mutual_information(dataset, target, task_type, drop_columns, bins=None, labels=None):
    """
    Calculate and visualize Mutual Information for dataset features.
    """
    # Drop unnecessary columns
    dataset = dataset.drop(drop_columns, axis=1)

    # Discretize target for classification tasks if bins are provided
    if task_type == 'classification' and bins is not None and labels is not None:
        target = pd.cut(target, bins=bins, labels=labels)

    # Choose mutual information function based on task type
    if task_type == 'classification':
        mi = mutual_info_classif(dataset, target, random_state=42)
    elif task_type == 'regression':
        mi = mutual_info_regression(dataset, target, random_state=42)
    else:
        raise ValueError("Invalid task_type. Use 'classification' or 'regression'.")

    # Create a Pandas Series of MI scores
    mi_scores = pd.Series(mi, index=dataset.columns)
    mi_scores = mi_scores.sort_values(ascending=False)

    # Plot Mutual Information Scores
    plt.figure(figsize=(10, 6))
    mi_scores.plot(kind='bar', color='skyblue')
    plt.title('Mutual Information Scores', fontsize=14)
    plt.ylabel('Mutual Information', fontsize=12)
    plt.xlabel('Features', fontsize=12)
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

    return mi_scores


drop_columns = ['latitude', 'longitude', 'local_time', 'state', 'country', 'no', 'nh3']

# For classification
bins = [0, 50, 100, 150, 200, float('inf')]
labels = [1, 2, 3, 4, 5]
calculate_mutual_information(copy.drop('aqi', axis=1), copy['aqi'], 'classification', drop_columns, bins=bins, labels=labels)

In [ ]:
# Regression mutual information without (NO, NH3).
calculate_mutual_information(copy.drop('aqi', axis=1), copy['aqi'], 'regression', drop_columns, bins=bins, labels=labels)

In [ ]:
# MI Classification with (NO, NH3)
drop_columns = ['latitude', 'longitude', 'local_time', 'state', 'country']

# For classification
bins = [0, 50, 100, 150, 200, float('inf')]
labels = [1, 2, 3, 4, 5]
calculate_mutual_information(copy.drop('aqi', axis=1), copy['aqi'], 'classification', drop_columns, bins=bins, labels=labels)

In [ ]:
# MI Regression with (NO, NH3)
calculate_mutual_information(copy.drop('aqi', axis=1), copy['aqi'], 'regression', drop_columns, bins=bins, labels=labels)

### Result:
The correlation and MI showes poor relationship with each features and we will use it again in `Feature Engineering` too. There are many steps remain to we make a decision about poor relationship right now just be patience and drink a cap of coffee or tea☕.<br>
Please be patience and continue the journey with us. 😉😊😊

### Handling Duplicate values.

In [ ]:
# again checking for duplication
copy.duplicated().sum()

In [ ]:
# checking duplicate records
copy[copy.duplicated()]

In [ ]:
# checking duplicate without some features
drop = ['country', 'state', 'aqi', 'local_time', 'latitude', 'longitude']
copy.drop(drop, axis=1).duplicated().sum()

In [ ]:
# checking duplicate as dataframe
copy[copy.duplicated()]

In [ ]:
# We need further analysis of duplication using in every feature
# To do this we will create a function named duplicate and process the duplication
def duplicate(dataset):
  duplication = {} # saving the duplicated as key is the name of feature and value is amount of duplication

  for feature in dataset.columns:
    duplication[feature] = dataset.duplicated(subset=[feature]).count()

  return duplication

# let's check now
res = duplicate(copy)
for key, val in res.items():
  print(f"Total {key} duplicate values: {val}")

In [ ]:
copy.duplicated(subset=['co'])

In [ ]:
du = copy[copy.duplicated(keep=False)]
du

In [ ]:
dup = copy.duplicated()
dup

In [ ]:
# checking only duplicate index
# this process is to make sure myself
def duplicate(dataset):
  dic = {}
  count = 0
  for index, du in enumerate(dataset):
    if du == True:
      count += 1
      dic[index] = du

  return dic, count

res, total = duplicate(dup)

print(res)
print(total)

In [ ]:
copy[:15]

In [ ]:
# time to remove duplicate records
copy.drop_duplicates(inplace=True)

In [ ]:
# checking the duplicatin to make sure everything goes smoothly
copy[data.duplicated()]

#### Distribution Checkup:
As you know we have checked some distribution checkup previously to fimilarize ourself with the data. But here we are going to check:
* Normal Distribution.
* Density.
* Statistical Analysis with Density.
* Shapiro Wilk.
* QQ Plot for Normalization Checkup.
* Log-Normal, Exponential and Gamma in one plot.

Hope to Enjoy 😎😎😎.

In [ ]:
# Checking distribution of data
def normal_dis(dataset):
  # Drop unwanted features
  drop = ['country', 'state', 'aqi', 'latitude', 'longitude', 'local_time']
  dataset = dataset.drop(drop, axis=1)

  # visualizing each feature using PDF
  for feature in dataset.columns:
    fig = px.histogram(dataset[feature], x=f"{feature}", title=f"{feature} Distribution")
    fig.show()
  return
normal_dis(copy)

In [ ]:
import plotly.graph_objects as go
from scipy.stats import norm
import plotly.io as pio
pio.renderers.default = "colab"

# Dropping unnecessary columns if any
normal_feature = copy.drop(['latitude', 'longitude', 'country', 'state', 'local_time', 'aqi'], axis=1, errors='ignore')

# Loop through features to plot statistical analysis using Plotly
for feature in normal_feature.columns:
    feature_data = normal_feature[feature].dropna()

    # Calculate key statistics
    mean = feature_data.mean()
    median = feature_data.median()
    std = feature_data.std()
    variance = feature_data.var()

    # Create the histogram and KDE plot with Plotly
    hist_data = go.Histogram(
        x=feature_data,
        histnorm='density',
        name='Histogram',
        opacity=0.7,
        marker=dict(color='steelblue'),
        nbinsx=30
    )

    # Calculate the normal distribution curve
    x_range = np.linspace(min(feature_data), max(feature_data), 100)
    y_range = norm.pdf(x_range, mean, std)

    normal_curve = go.Scatter(
        x=x_range,
        y=y_range,
        mode='lines',
        name='Normal Distribution',
        line=dict(color='purple', width=2)
    )

    # Add vertical lines for Mean, Median, and +/- 1 Standard Deviation
    mean_line = go.Scatter(
        x=[mean, mean],
        y=[0, max(y_range) * 1.1],  # Extend the line along the y-axis
        mode='lines',
        name=f'Mean = {mean:.2f}',
        line=dict(color='red', dash='dash', width=3)
    )

    median_line = go.Scatter(
        x=[median, median],
        y=[0, max(y_range) * 1.1],  # Extend the line along the y-axis
        mode='lines',
        name=f'Median = {median:.2f}',
        line=dict(color='green', dash='dash', width=3)
    )

    std_plus_line = go.Scatter(
        x=[mean + std, mean + std],
        y=[0, max(y_range) * 1.1],  # Extend the line along the y-axis
        mode='lines',
        name=f'+1 SD = {mean + std:.2f}',
        line=dict(color='orange', dash='dash', width=3)
    )

    std_minus_line = go.Scatter(
        x=[mean - std, mean - std],
        y=[0, max(y_range) * 1.1],  # Extend the line along the y-axis
        mode='lines',
        name=f'-1 SD = {mean - std:.2f}',
        line=dict(color='orange', dash='dash', width=3)
    )

    # Add annotations for variance
    variance_text = go.layout.Annotation(
        x=mean + 3 * std,
        y=max(y_range) * 0.05,
        text=f'Variance = {variance:.2f}',
        showarrow=True,
        font=dict(size=12, color='purple'),
        align='left'
    )

    # Create the layout with static range
    layout = go.Layout(
        title=f'Statistical Analysis of {feature}',
        xaxis=dict(
            title=feature,
            showgrid=True,
        ),
        yaxis=dict(
            title='Density',
            rangemode='tozero',  # Ensure the y-axis starts from zero
            showgrid=True,
        ),
        showlegend=True,
        template="plotly_dark",  # Optional, change to "plotly" for a lighter theme
        annotations=[variance_text]
    )

    # Create the figure and show
    fig = go.Figure(
        data=[hist_data, normal_curve, mean_line, median_line, std_plus_line, std_minus_line],
        layout=layout
    )

    # Show the plot
    fig.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

# Dropping unnecessary columns
normal_feature = copy.drop(['latitude', 'longitude', 'country', 'state', 'local_time', 'aqi'], axis=1, errors='ignore')

# Loop through features to plot statistical analysis using Seaborn and Matplotlib
for feature in normal_feature.columns:
    plt.figure(figsize=(10, 6))

    # Get feature data and drop null values
    feature_data = normal_feature[feature].dropna()

    # Calculate key statistics
    mean = feature_data.mean()
    median = feature_data.median()
    std = feature_data.std()
    variance = feature_data.var()

    # Plot the density curve with Seaborn
    sns.histplot(feature_data, kde=True, color='steelblue', stat='density', label='Density Curve')

    # Plot mean, median, and standard deviation lines
    plt.axvline(mean, color='red', linestyle='dashed', linewidth=3, label=f'Mean = {mean:.2f}')
    plt.axvline(median, color='green', linestyle='dashed', linewidth=3, label=f'Median = {median:.2f}')
    plt.axvline(mean + std, color='orange', linestyle='dashed', linewidth=3, label=f'+1 SD = {mean + std:.2f}')
    plt.axvline(mean - std, color='orange', linestyle='dashed', linewidth=3, label=f'-1 SD = {mean - std:.2f}')


    # Title and labels
    plt.title(f'Statistical Analysis of {feature}', fontsize=16)
    plt.xlabel(feature, fontsize=14)
    plt.ylabel('Density', fontsize=14)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)

    # Show plot
    plt.show()

Probability Density Estimation mostly showes un-normal distribution but still we want to make sure about it and check Shapiro Milk method.

In [ ]:
from scipy.stats import shapiro

def shapiro_test(dataset):
  # Perform the Shapiro-Wilk and saving each feature result in results
  results = {}

  # drop unwanted feature
  drop = ['aqi', 'local_time','longitude', 'latitude', 'country', 'state']
  dataset = dataset.drop(drop, axis=1)

  for feature in dataset.columns:
      feature_data = dataset[feature].dropna()  # Drop missing values
      stat, p_value = shapiro(feature_data)
      results[feature] = {'Statistic': stat, 'P-Value': p_value}


  # Interpretation
  for feature, result in results.items():
    p_value = result['P-Value']
    if p_value > 0.05:
        print(f"{feature}: Likely Normally Distributed (p = {p_value:.4f})")
    else:
        print(f"{feature}: Not Normally Distributed (p = {p_value:.4f})")

shapiro_test(copy)

In [ ]:
from scipy.stats import kstest, norm

def Kolmogorov_Smirnov(dataset):
  # Perform the K-S test for each feature
  results = {}

  # drop unwanted feature
  drop = ['aqi', 'local_time','longitude', 'latitude', 'country', 'state']
  dataset = dataset.drop(drop, axis=1)


  for feature in dataset.columns:
      feature_data = dataset[feature].dropna()  # Drop null values
      # Standardize the data (mean=0, std=1) for comparison with normal distribution
      standardized_data = (feature_data - feature_data.mean()) / feature_data.std()

      # Perform K-S test against the standard normal distribution
      stat, p_value = kstest(standardized_data, 'norm')
      results[feature] = {'Statistic': stat, 'P-Value': p_value}


  # Interpretation
  for feature, result in results.items():
      p_value = result['P-Value']
      if p_value > 0.05:
        print(f"{feature}: Likely Normally Distributed (p = {p_value:.4f})")
      else:
        print(f"{feature}: Not Normally Distributed (p = {p_value:.4f})")

Kolmogorov_Smirnov(copy)

In [ ]:
from scipy import stats

def qq_plot(dataset):
  # drop unwanted features
  drop = ['aqi', 'local_time','longitude', 'latitude', 'country', 'state']
  dataset = dataset.drop(drop, axis=1)

  for feature in dataset.columns:
      plt.figure(figsize=(10, 6))

      # Q-Q Plot
      (osm, osr), (slope, intercept, r) = stats.probplot(data[feature], dist="norm", plot=None)
      plt.scatter(osm, osr, color='steelblue', edgecolor='black', alpha=0.7, label="Data Points")
      plt.plot(osm, slope * np.array(osm) + intercept, color='red', lw=2, label=f"Fit Line (R² = {r**2:.2f})")

      # Annotations for key regions
      plt.axvline(0, color='gray', linestyle='--', alpha=0.7)  # Mean
      plt.axhline(0, color='gray', linestyle='--', alpha=0.7)  # Theoretical mean

      # Add legend
      plt.legend(fontsize=12)

      # Labels and Title
      plt.title(f"Q-Q Plot for {feature}", fontsize=16)
      plt.xlabel("Theoretical Quantiles", fontsize=14)
      plt.ylabel("Sample Quantiles", fontsize=14)
      plt.grid(True, linestyle="--", alpha=0.6)
      plt.show()

qq_plot(copy)

### Result
Based on description above we are now sure that our data is not normal distributed and can check other distributions.

In [ ]:
import scipy.stats as stats

def overlaping_pdf(dataset):
    # Drop unwanted features
    drop = ['aqi', 'local_time', 'longitude', 'latitude', 'country', 'state']
    dataset = dataset.drop(columns=drop, errors='ignore')

    # Plot histogram of the data for each feature
    for feature in dataset.columns:
        plt.figure(figsize=(10, 6))

        # Get feature data
        feature_data = dataset[feature].dropna()

        # Plot histogram of the data
        sns.histplot(feature_data, kde=False, stat='density', bins=30, color='steelblue', label='Data')

        # Prepare x-axis values for PDF plotting
        x = np.linspace(feature_data.min(), feature_data.max(), 1000)

        # Distributions to fit
        distributions = {
            'Normal': stats.norm,
            'Log-Normal': stats.lognorm,
            'Exponential': stats.expon,
            'Gamma': stats.gamma
        }

        # Fit and plot PDFs for different distributions
        for name, dist in distributions.items():
            try:
                if name == 'Log-Normal':
                    # Filter positive values for log-normal
                    positive_data = feature_data[feature_data > 0]
                    if len(positive_data) > 0:  # Ensure there are positive values
                        params = dist.fit(positive_data)
                        plt.plot(x, dist.pdf(x, *params), label=f'{name} (fit)', lw=2)
                else:
                    # Fit other distributions
                    params = dist.fit(feature_data)
                    plt.plot(x, dist.pdf(x, *params[:-2], loc=params[-2], scale=params[-1]), label=f'{name} (fit)', lw=2)
            except Exception as e:
                print(f"Could not fit {name} for {feature}: {e}")

        # Labels, title, and legend
        plt.title(f'Comparing Distributions for {feature}', fontsize=16)
        plt.xlabel('Value', fontsize=14)
        plt.ylabel('Density', fontsize=14)
        plt.legend(fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.6)

        # Show the plot
        plt.show()

# Call the function
overlaping_pdf(copy)

In [ ]:
# installing profiling
# !pip install ydata-profiling

In [ ]:
# checking full data report using profiling
from ydata_profiling import ProfileReport

profile = ProfileReport(copy, title='Dataset Report')
profile

### Stationarity in Time Series:
Before immersion in transformation, it is important that we have to check the stationarity of our features and based on that we will make a decision about transformation. <br>
**Brief Info about Stationarity:**<br>
Stationarity in time series refers to a property where the statistical characteristics of the series, such as mean, variance, and autocorrelation, remain constant over time. This means that the behavior of the series does not depend on the time at which it is observed, and its unconditional joint probability distribution does not change when shifted in time.<br>
You can check about stationarity and non-stationarity in [here](https://medium.com/@ritusantra/stationarity-in-time-series-887eb42f62a9).<br>
For further information please check [here](https://www.analyticsvidhya.com/blog/2021/06/statistical-tests-to-check-stationarity-in-time-series-part-1/).

In [ ]:
from statsmodels.tsa.stattools import adfuller

def check_stationarity(dataset, time_col):
    # Ensure the time column is in datetime format
    dataset[time_col] = pd.to_datetime(dataset[time_col], errors='coerce', dayfirst=True,  format='%Y-%m-%d %H:%M:%S')

    # Set time column as index for time series operations
    dataset = dataset.set_index(time_col)

    # Iterate through each feature
    for feature in dataset.columns:
        print(f"\nAnalyzing Feature: {feature}")
        print("-" * 40)

        # Plot the time series
        plt.figure(figsize=(12, 6))
        plt.plot(dataset[feature], color='blue', label=f'{feature} Time Series')
        plt.title(f'{feature} Over Time', fontsize=16)
        plt.xlabel('Time', fontsize=14)
        plt.ylabel(feature, fontsize=14)
        plt.legend(fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.show()

        # Perform the Augmented Dickey-Fuller (ADF) Test
        print("Performing ADF Test...")
        result = adfuller(dataset[feature].dropna())  # Drop NaNs to avoid errors
        print(f"ADF Statistic: {result[0]}")
        print(f"p-value: {result[1]}")
        print(f"Critical Values: {result[4]}")

        # Check stationarity
        if result[1] <= 0.05:
            print("Result: The feature is stationary (p-value <= 0.05).")
        else:
            print("Result: The feature is non-stationary (p-value > 0.05).")

# drop unwanted features
drop = ['country', 'state', 'aqi', 'longitude', 'latitude']
dataset = copy.drop(drop, axis=1)
check_stationarity(dataset, 'local_time')

### Result:
All the feature are stationary.

### Outliers Checkup:
We are going to check outliers using boxplot and IQR

In [ ]:
def outliers(dataset):
  # unwanted features
  drop = ['country', 'state', 'longitude', 'latitude', 'aqi', 'local_time']
  dataset = dataset.drop(drop, axis=1)
  for feature in dataset.columns:
    # Calculate Q1 (25th percentile), Q3 (75th percentile) and IQR (Interquartile Range)
    Q1 = dataset[feature].quantile(0.25)
    Q3 = dataset[feature].quantile(0.75)
    IQR = Q3 - Q1

    # Define the lower and upper bounds for detecting outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Displaying lower and upper bounds
    print(f"Lower Bound: {lower_bound}")
    print(f"Upper Bound: {upper_bound}")

    # Identifying outliers and normal values
    dataset['outlier'] = np.where((dataset[feature] < lower_bound) | (dataset[feature] > upper_bound), 'Outlier', 'Normal')

    # Create the interactive boxplot using Plotly
    fig = px.box(dataset,
                x=feature,
                color="outlier",
                category_orders={"outlier": ["Normal", "Outlier"]},
                labels={feature: f"{feature} Value", "outlier": "Outlier Status"},
                title="Outlier Detection using Boxplot",
                points="all"  # Show all points, including outliers
                )

    # Customize the plot to make it clearer
    fig.update_traces(marker=dict(size=8),  # Increase point size for better visibility
                      boxmean='sd')  # Show the mean as a line


    fig.show()

outliers(copy)

### Transformation:
Based on different types of distribution, Normal checkup distribution we have done. It showed that our data is not normal distributed, in addition, we have right or positive skew distribution that requires to be transformed but as we checked stationary, our dataset is stationary and does not requires transformation. But as our goal is to remove outliers, building a prediction and classification algorithm of neural network with the combination of reinforcement learning, we need to use transformation to robust the data.<br>
`Different types of Transformation:`
* Box cox transformation
* Yeo-Johnson Transformation
* log-normal Transformation
* ...
<br>Each of these transformation is used for specific purpose. but as we have knowledge of our dataset and we have zero, negative and positive values, right skewed and time series dataset. One of best option for us is using `Yeo-Johnson Transformation`.<br>
please check for more details [here](https://towardsdatascience.com/types-of-transformations-for-better-normal-distribution-61c22668d3b9).


In [ ]:
from sklearn.preprocessing import PowerTransformer

def power_transformation(dataset):
    # Unwanted features to drop
    drop = ['country', 'state', 'longitude', 'latitude', 'local_time', 'aqi']
    dropped_features = dataset[drop]  # Save the dropped features to merge later
    dataset = dataset.drop(drop, axis=1)

    # Apply Yeo-Johnson transformation to each feature individually
    transformed_features = {}
    for feature in dataset.columns:
        transformer = PowerTransformer(method='yeo-johnson', standardize=False)
        transformed_features[feature] = transformer.fit_transform(dataset[[feature]]).flatten()

    # Create a new DataFrame with transformed features
    transformed_data = pd.DataFrame(transformed_features)

    # Join the dropped features back into the dataset
    final_dataset = pd.concat([dropped_features.reset_index(drop=True), transformed_data], axis=1)

    # Print some statistics to compare
    for col in dataset.columns:
        print(f"Feature: {col}")
        print(f" - Skewness before: {dataset[col].skew():.2f}")
        print(f" - Skewness after: {transformed_data[col].skew():.2f}")
        print(f" - Mean before: {dataset[col].mean():.2f}")
        print(f" - Mean after: {transformed_data[col].mean():.2f}")
        print()

    return final_dataset  # Return the transformed dataset

transformed_data = power_transformation(copy)
transformed_data

In [ ]:
# Visualizing the distribution
def after_trans(dataset):
  # unwanted feature
  drop = ['country', 'state', 'longitude', 'latitude', 'aqi', 'local_time']
  dataset = dataset.drop(drop, axis=1)

  for feature in dataset.columns:
    fig = px.histogram(dataset[feature], x=feature)
    fig.show()

  return

after_trans(transformed_data)

Still we are facing with huge amount of outliers

In [ ]:
def check_and_remove_outliers(dataset):
    # Unwanted features to drop temporarily
    drop = ['country', 'state', 'latitude', 'longitude', 'local_time', 'aqi']
    unwanted_data = dataset[drop]  # Store unwanted features
    numeric_data = dataset.drop(drop, axis=1)  # Keep only numeric features

    # Store all outlier indices
    outliers_indices = set()

    # Iterate over each numeric feature in the dataset
    for feature in numeric_data.columns:
        if pd.api.types.is_numeric_dtype(numeric_data[feature]):
            Q1 = numeric_data[feature].quantile(0.25)  # First quartile (25th percentile)
            Q3 = numeric_data[feature].quantile(0.75)  # Third quartile (75th percentile)
            IQR = Q3 - Q1  # Interquartile range

            # Calculate bounds for outliers
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # Find outlier indices and add them to the set
            outliers = numeric_data[(numeric_data[feature] < lower_bound) | (numeric_data[feature] > upper_bound)]
            outliers_indices.update(outliers.index.tolist())

            # Print summary
            print(f"Feature '{feature}':")
            print(f" - Lower Bound: {lower_bound:.2f}, Upper Bound: {upper_bound:.2f}")
            print(f" - Outliers Found: {len(outliers)}")

    # Remove outliers by index from both datasets
    numeric_data_cleaned = numeric_data.drop(index=outliers_indices, errors='ignore')
    unwanted_data_cleaned = unwanted_data.drop(index=outliers_indices, errors='ignore')

    # Reset the index for both datasets
    numeric_data_cleaned.reset_index(drop=True, inplace=True)
    unwanted_data_cleaned.reset_index(drop=True, inplace=True)

    # Concatenate the numeric data with the unwanted features
    final_dataset = pd.concat([unwanted_data_cleaned, numeric_data_cleaned], axis=1)

    return final_dataset, sorted(outliers_indices)


# Example usage
cleaned_data, removed_outliers = check_and_remove_outliers(transformed_data)

# Print the cleaned data and removed outliers
print("Cleaned Dataset Shape:", cleaned_data.shape)
print("Number of Outliers Removed:", len(removed_outliers))

In [ ]:
cleaned_data

In [ ]:
def after_outliers(dataset):
    # Unwanted features to drop temporarily
    drop = ['country', 'state', 'latitude', 'longitude', 'local_time', 'aqi']
    unwanted_data = dataset[drop]  # Store unwanted features
    numeric_data = dataset.drop(drop, axis=1)  # Keep only numeric features

    # Iterate over each numeric feature in the dataset
    for feature in numeric_data.columns:
        # Calculate the upper and lower limits
        Q1 = numeric_data[feature].quantile(0.25)
        Q3 = numeric_data[feature].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR

        # Create arrays of Boolean values indicating the outlier rows
        upper_array = np.where(numeric_data[feature] > upper)[0]
        lower_array = np.where(numeric_data[feature] < lower)[0]

        # Print results
        print(f"Feature: {feature}")
        print(f" - Upper Bound Outliers Indices: {upper_array}")
        print(f" - Lower Bound Outliers Indices: {lower_array}")
        print(f" - Total Outliers: {len(upper_array) + len(lower_array)}\n")

after_outliers(cleaned_data)

In [ ]:
# checking graphs after outliers dropped
after_trans(cleaned_data)

In [ ]:
# The total affect of outliers in a dataset
print(f"Total percentage we lost after Outliers: {12048/len(cleaned_data) * 100:.2f}%")

### Result
As you can see above we lost huge amount of data after normal outlier checkup and dropout. In addition, still we have outliers after cleaning outliers and this showes that we can not go with normal outliers checkup and removing.<br>
**To handle this:** we need to use Robust Scaler method it won't all outliers but it will manage it to model can handle it. Also keep in mind that those outliers important data and based on countries dangrous air quality outliers happend.

In [ ]:
from sklearn.preprocessing import RobustScaler

def robust_scaler(dataset):
  # Drop unwanted features for scaling process
  drop = ['country', 'state', 'longitude', 'latitude', 'local_time', 'aqi']
  X = dataset.drop(drop, axis=1)
  y = dataset['aqi']  # Target variable (AQI)

  # Initialize RobustScaler
  scaler = RobustScaler()

  # Apply RobustScaler to each feature individually
  X_scaled = X.apply(lambda column: scaler.fit_transform(column.values.reshape(-1, 1)).flatten())

  # Reattach the unwanted features to the scaled dataset
  X_scaled['country'] = dataset['country']
  X_scaled['state'] = dataset['state']
  X_scaled['longitude'] = dataset['longitude']
  X_scaled['latitude'] = dataset['latitude']
  X_scaled['local_time'] = dataset['local_time']

  # Add the target variable 'aqi' back to the dataset
  X_scaled['aqi'] = y

  # Display the final dataset with both scaled and original columns
  print(X_scaled)

  return X_scaled

robust_data = robust_scaler(transformed_data)
robust_data

In [ ]:
# checking the histogram
after_trans(robust_data)

In [ ]:
# lets check total outliers again
def total_outliers(dataset):
  # Unwanted features to drop temporarily
  drop = ['country', 'state', 'latitude', 'longitude', 'local_time', 'aqi']
  unwanted_data = dataset[drop]  # Store unwanted features
  dataset = dataset.drop(drop, axis=1)  # Keep only numeric features

  # Quantile
  Q1 = dataset.quantile(0.25)
  Q3 = dataset.quantile(0.75)

  # Lower and Upper bound
  lower = Q1 - 1.5 * (Q3 - Q1)
  upper = Q3 + 1.5 * (Q3 - Q1)

  # checking for outliers
  outlier = ((dataset > upper) | (dataset < lower))

  print(outlier.sum())

total_outliers(robust_data)

In [ ]:
# checking the statistical analysis of before and after robust scaler
# Before Robust Scaler
transformed_data.describe()

In [ ]:
# After Robust Scaling
robust_data.describe()

### Result:
After long processing finally we have handled the outliers. But take note that I do not mean solving or making the outliers zero but we have transfored from huge differences to less in values but yes still there is problem in distribution and if we remove those ourtliers it would damage important data such countries like India, Pakistan, Iran, Afghanistan and some other countries records will be deleted. To avoid this we have used Robust Scaler and now with the sitution above described in statistical analysis I wishes it wount hurt to much the models and we will use some benificial activition fuction to solve it.
So with all these condition we wishes still be positive and bear to go for further analysis. 😊😊😊😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😃😃😃

In [ ]:
# checking full data report using profiling
from ydata_profiling import ProfileReport

profile = ProfileReport(clean_data, title='Dataset Report')
profile

### Next Step:
* Kabul local data has missmatch date format and required to standardize it.

In [ ]:
# Time to handle Afghanistan local time
def kabul_date_format(dataset):
    # Select rows where 'state' is 'Kabul'
    mask = dataset['state'] == 'Kabul'

    # Identify rows needing conversion (non-standard date format)
    needs_conversion = ~dataset.loc[mask, 'local_time'].str.match(r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$', na=False)

    # Convert non-standard dates to datetime format
    dataset.loc[mask & needs_conversion, 'local_time'] = pd.to_datetime(
        dataset.loc[mask & needs_conversion, 'local_time'],
        format='%d/%m/%Y',
        errors='coerce',
        dayfirst=True
    )

    # Add missing time details for converted dates
    dataset.loc[mask & needs_conversion, 'local_time'] = dataset.loc[mask & needs_conversion, 'local_time'].apply(
        lambda x: x.replace(hour=0, minute=0, second=0) if not pd.isnull(x) else x
    )

    return dataset

time_data = kabul_date_format(robust_data)

time_data

In [ ]:
# Checking wheather there is missing value occures during time convertion format
time_data[time_data['local_time'].isna()]

In [ ]:
# lets compare with exact row time format and then fill it
miss_time = data['local_time'].loc[54881]
time_data['local_time'] = time_data['local_time'].fillna(pd.Timestamp(miss_time))
time_data

In [ ]:
# lets check kabul local time
time_data[time_data['state'] == 'Kabul']['local_time'].value_counts()

### Result:
So far we have done these missions:
* Converting kabul time format to standard (ISO8601) format.


### **Feature Engineering:**
`Info:`
Feature engineering is a very important step in machine learning. Feature engineering refers to the process of designing artificial features into an algorithm. These artificial features are then used by that algorithm in order to improve its performance, or in other words, reap better results.<br>
**`Our Policy:`**<br>
In this section we are going to check these steps:
* Correlation and Covariance of features.
* PCA (Principal Component Analysis)
* Feature Selection
* Pairwise Interaction Terms
* Handle mostly zero value in a feature
* Rolling and Laging to create trend and seasonality.
* Balancing data based on countries using ARIMA
method
* clustring the data based on longitude and latitude
* Balancing the AQI using smoth method


In [ ]:
# lets go with the first step of Feature Engineering (Correlation and Covariance) :)
# This correlation and covarience do not support (NO, NH3) feature but the next one will
def corr_and_cov(dataset, drop):
    dataset = dataset.drop(drop, axis=1)

    # Compute correlation and covariance matrices
    corr = dataset.corr()
    cov = dataset.cov()

    # Create the plots
    plt.figure(figsize=(16, 6))

    # Correlation heatmap
    plt.subplot(1, 2, 1)  # 1 row, 2 columns, first plot
    sns.heatmap(corr, annot=True, cmap='coolwarm', cbar=True, square=True)
    plt.title('Data Correlation')

    # Covariance heatmap
    plt.subplot(1, 2, 2)  # 1 row, 2 columns, second plot
    sns.heatmap(cov, annot=True, cmap='viridis', cbar=True, square=True)
    plt.title('Data Covariance')

    # Adjust layout and show the plots
    plt.tight_layout()
    plt.show()

# Drop unwanted features
drop = ['latitude', 'longitude', 'local_time', 'state', 'country', 'no', 'nh3']
corr_and_cov(time_data, drop)

In [ ]:
# correlation and covarience with no and nh3
drop = ['latitude', 'longitude', 'local_time', 'state', 'country']
corr_and_cov(time_data, drop)

### Result:
Based on the description of features. (pm2.5 and pm10) have a strong correlation that indicate some common values. And also NH3 and NO has less effeciency on AQI. We need to check again using PCA.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def perform_pca(dataset, n_components=None):
    # Unwanted features to drop temporarily
    drop = ['country', 'state', 'latitude', 'longitude', 'local_time', 'aqi']
    unwanted_data = dataset[drop]  # Store unwanted features
    numeric_data = dataset.drop(drop, axis=1)  # Numeric features for PCA

    # Standardize the numeric data for PCA
    scaler = StandardScaler()
    standardized_data = scaler.fit_transform(numeric_data)

    # Perform PCA
    pca = PCA(n_components=n_components)
    principal_components = pca.fit_transform(standardized_data)

    # Create a DataFrame for the principal components
    pca_columns = [f"PC{i+1}" for i in range(principal_components.shape[1])]
    pca_df = pd.DataFrame(data=principal_components, columns=pca_columns)

    # Combine original dataset (unscaled numeric data) with PCA components and unwanted data
    final_dataset = pd.concat([dataset.reset_index(drop=True), pca_df], axis=1)

    # Variance explained by each component
    explained_variance_ratio = pca.explained_variance_ratio_

    # Plot 1: Explained variance ratio
    plt.figure(figsize=(8, 6))
    plt.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, alpha=0.7, align='center', label='Explained Variance')
    plt.step(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio.cumsum(), where='mid', color='red', label='Cumulative Variance')
    plt.xlabel('Principal Components')
    plt.ylabel('Explained Variance Ratio')
    plt.title('PCA Explained Variance')
    plt.legend(loc='best')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

    # Plot 2: Feature contributions to each principal component
    feature_names = numeric_data.columns  # Original feature names
    pca_components = pca.components_  # PCA component weights

    plt.figure(figsize=(12, 6))
    colors = ['blue', 'green', 'orange', 'red', 'purple']  # Different colors for each PC

    for i, pc in enumerate(pca_columns):
        plt.bar(feature_names, pca_components[i], color=colors[i % len(colors)], alpha=0.7, label=pc)

    plt.title('Feature Contributions to Each Principal Component', fontsize=16)
    plt.xlabel('Features', fontsize=12)
    plt.ylabel('Contribution', fontsize=12)
    plt.xticks(rotation=45, fontsize=10)
    plt.legend(title='Principal Components', fontsize=10)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

    return final_dataset, explained_variance_ratio

pca_result, explained_variance = perform_pca(time_data, n_components=5)


print("\nExplained Variance Ratio:")
print(explained_variance)
print(sum(explained_variance))

### Result:
As we can see the varience ratio indicate how each pc's contribution to the dataset and the first pc has the most highest values of data inside. In addition totally the five pcs can include `91%` of the data and we lost `9%` of the data, so we can increase the principle component or just continue with the feature Engineering. Next, second graph showes how each feature contribute to the pcs.<br>
We will go further steps of `Feature Engineering` and after that make decision about PCA.


### Next steps:
* Adding features based on Years, Months, Days, Hour and Minutes.


In [ ]:
# Now lets handle local_time
# Add features for year, month, day, hour, and minute
def add_time_features(dataset):
    dataset['local_time'] = pd.to_datetime(dataset['local_time'], errors='coerce')

    # Extract and add features
    dataset['year'] = dataset['local_time'].dt.year
    dataset['month'] = dataset['local_time'].dt.month
    dataset['day'] = dataset['local_time'].dt.day
    dataset['hour'] = dataset['local_time'].dt.hour
    dataset['dayofweek'] = dataset['local_time'].dt.dayofweek

    return dataset

time_data = add_time_features(time_data)

time_data.head()

In [ ]:
# Time to drop local_time feature
time_data.drop('local_time', axis=1, inplace=True)
time_data

### Time to handle imbalance data and balance it based on countries.
why selecting countries?<br>
Well mostly we do not have sensors in every state or province of countries to we select based on their states. This is why we have select countries and in some countries just there is one or sensors like Afghanistan. So, that is why we have selected based on countries.

In [ ]:
# lets check again the countries records in dataset
np.array(time_data['country'].value_counts())

In [ ]:
# its percentage
np.array(time_data['country'].value_counts() / len(time_data))

In [ ]:
# Save time data
# time_data.to_csv("/content/drive/MyDrive/AFG-FTL-Capstone-Project/time_data.csv", index=False)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Function to generate synthetic data for a given country's data
def generate_synthetic_data(country_data, target_count=350):
    """
    Generate synthetic time-series data for a given country's data using ARIMA.
    """
    current_count = len(country_data)

    if current_count >= target_count:
        return country_data  # No need to generate additional data

    # Calculate the number of records to generate
    additional_count = target_count - current_count

    # Generate synthetic data for each feature
    synthetic_records = []
    for feature in country_data.columns.drop(['country', 'state', 'longitude', 'latitude', 'year', 'month', 'day', 'hour', 'dayofweek']):
        # Fit ARIMA on the existing data
        model = ARIMA(country_data[feature], order=(1, 1, 1))  # Adjust order if necessary
        model_fit = model.fit()

        # Generate synthetic values
        forecast = model_fit.forecast(steps=additional_count)

        # Ensure proper types for specific features
        if feature == 'aqi':
            forecast = np.round(forecast).astype(int)  # Ensure aqi remains integer
        synthetic_records.append(forecast)

    # Combine synthetic data into a DataFrame
    synthetic_data_df = pd.DataFrame(synthetic_records).T
    synthetic_data_df.columns = country_data.columns.drop(['country', 'state', 'longitude', 'latitude', 'year', 'month', 'day', 'hour', 'dayofweek'])

    # Assign fixed values for `country`, `state`, `longitude`, `latitude`
    synthetic_data_df['country'] = country_data['country'].iloc[0]
    synthetic_data_df['state'] = country_data['state'].iloc[0]
    synthetic_data_df['longitude'] = country_data['longitude'].iloc[0]
    synthetic_data_df['latitude'] = country_data['latitude'].iloc[0]

    # Assign timestamp to synthetic data
    synthetic_data_df['year'] = country_data['year'].iloc[-1]
    synthetic_data_df['month'] = country_data['month'].iloc[-1]
    synthetic_data_df['day'] = country_data['day'].iloc[-1] + np.arange(1, additional_count + 1) // 24
    synthetic_data_df['hour'] = (country_data['hour'].iloc[-1] + np.arange(1, additional_count + 1)) % 24
    synthetic_data_df['dayofweek'] = country_data['dayofweek'].iloc[-1]

    # Combine original and synthetic data
    balanced_data = pd.concat([country_data.reset_index(drop=True), synthetic_data_df])

    return balanced_data

# Main balancing function
def balance_dataset_with_arima(dataset, target_count=320):
    """
    Balance the dataset by generating synthetic records for underrepresented countries.
    """
    # Group by country and count records
    country_counts = dataset['country'].value_counts()

    # Identify underrepresented countries
    underrepresented_countries = country_counts[country_counts < target_count].index

    # Balance the dataset
    balanced_dataset = []
    for country in underrepresented_countries:
        country_data = dataset[dataset['country'] == country]
        balanced_country_data = generate_synthetic_data(country_data, target_count=target_count)
        balanced_dataset.append(balanced_country_data)

    # Include countries already having enough records
    sufficient_data_countries = dataset[~dataset['country'].isin(underrepresented_countries)]
    balanced_dataset.append(sufficient_data_countries)

    # Combine all balanced data
    balanced_dataset = pd.concat(balanced_dataset)
    balanced_dataset.reset_index(drop=True, inplace=True)

    return balanced_dataset

# Balance the dataset
balanced_dataset = balance_dataset_with_arima(time_data, target_count=320)

# Validate record counts
balanced_country_counts = balanced_dataset['country'].value_counts()
print(balanced_country_counts)


In [ ]:
# Checking tha balance of dataset of after implementing ARIMA
np.array(balanced_dataset['country'].value_counts())

In [ ]:
# Checking the percentage of each country:
np.array(balanced_dataset['country'].value_counts() / len(balanced_dataset))

In [ ]:
# Checking the duplicated values agian
balanced_dataset.duplicated().sum()

Dropping country and state features of countries and save the dataset.

In [ ]:
# Right now we do not need country and state features. So lets drop them.
balanced_dataset.drop(['country', 'state'], axis=1, inplace=True)

In [ ]:
# Saving this new dataset
# balanced_dataset.to_csv("/content/drive/MyDrive/AFG-FTL-Capstone-Project/balanced_dataset.csv", index=False)

In [ ]:
# Now lets import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
%matplotlib inline

In [ ]:
# Read data
balanced_dataset = pd.read_csv("/content/drive/MyDrive/AFG-FTL-Capstone-Project/balanced_dataset.csv")
balanced_dataset

Checking Correlation and Covariance.

In [ ]:
# lets check correlation and covarince.
def corr_and_cov(dataset, drop):
    dataset = dataset.drop(drop, axis=1)
    # Compute correlation and covariance matrices
    corr = dataset.corr()
    cov = dataset.cov()

    # Create the plots
    plt.figure(figsize=(20, 10))

    # Correlation heatmap
    plt.subplot(1, 2, 1)  # 1 row, 2 columns, first plot
    sns.heatmap(corr, annot=True, cmap='coolwarm', cbar=True, square=True)
    plt.title('Data Correlation')

    # Covariance heatmap
    plt.subplot(1, 2, 2)  # 1 row, 2 columns, second plot
    sns.heatmap(cov, annot=True, cmap='viridis', cbar=True, square=True)
    plt.title('Data Covariance')

    # Adjust layout and show the plots
    plt.tight_layout()
    plt.show()

drop = ['longitude', 'latitude', 'year', 'month', 'day', 'hour', 'dayofweek']
corr_and_cov(balanced_dataset, drop)

In [ ]:
# Lets check without nh3 and no
drop = ['longitude', 'latitude', 'year', 'month', 'day', 'hour', 'dayofweek', 'nh3', 'no']
corr_and_cov(balanced_dataset, drop)

In [ ]:
# Lets check without nh3, p,2.5 and no
drop = ['longitude', 'latitude', 'year', 'month', 'day', 'hour', 'dayofweek', 'nh3', 'no', 'pm10']
corr_and_cov(balanced_dataset, drop)

**Feature Importance Using RandomForest**

In [ ]:
# Implementing feature importance without nh3, no
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

def feature_importance_with_rf(dataset, drop):
  X = dataset.drop(drop, axis=1)
  y = dataset['aqi']

  # Split data into training and testing sets
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

  # Initialize a Random Forest Classifier (you can also use RandomForestRegressor for regression tasks)
  rf = RandomForestRegressor(n_estimators=100, random_state=42)

  # Train the Random Forest model
  rf.fit(X_train, y_train)

  # Get feature importance scores
  importances = rf.feature_importances_

  # Create a DataFrame to display feature importance
  feature_importance_df = pd.DataFrame({
      'Features': X.columns,
      'Importance': importances
  })

  # Sort by importance
  feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

  # Plot feature importance
  plt.figure(figsize=(10,6))
  plt.barh(feature_importance_df['Features'], feature_importance_df['Importance'])
  plt.xlabel('Importance')
  plt.title('Feature Importance')
  plt.show()

drop = ['latitude', 'longitude', 'nh3', 'no', 'aqi']
feature_importance_with_rf(time_data, drop)

In [ ]:
# Feature importance with no and nh3
drop = ['latitude', 'longitude', 'aqi']
feature_importance_with_rf(time_data, drop)

### Feature Importance with MI
In this step we are going to check Classification with and without (NH3, NO)

In [ ]:
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression

def calculate_mutual_information(dataset, target, task_type, drop_columns, n_bins=None, labels=None):
    """
    Calculate and visualize Mutual Information for dataset features with robust binning.
    """
    # Ensure dataset and target have the same number of rows
    if len(dataset) != len(target):
        raise ValueError("Dataset and target must have the same number of rows.")

    # Drop unnecessary columns
    dataset = dataset.drop(drop_columns, axis=1)

    # Dynamically adjust bins based on target distribution if classification task
    if task_type == 'classification' and n_bins is not None:
        target_min, target_max = target.min(), target.max()
        bins = np.linspace(target_min, target_max, n_bins + 1)  # Dynamic bin edges
        print(f"Dynamic bins: {bins}")  # Debug bin edges
        if labels is None:
            labels = range(1, n_bins + 1)  # Default labels as integers
        target = pd.cut(target, bins=bins, labels=labels, include_lowest=True)

        # Check and handle NaN values in target after binning
        if target.isna().sum() > 0:
            print("Warning: NaN values found in target after binning. Dropping invalid rows.")
            valid_indices = target.notna()
            dataset = dataset[valid_indices]
            target = target[valid_indices]

    # Choose mutual information function based on task type
    if task_type == 'classification':
        mi = mutual_info_classif(dataset, target, random_state=42)
    elif task_type == 'regression':
        mi = mutual_info_regression(dataset, target, random_state=42)
    else:
        raise ValueError("Invalid task_type. Use 'classification' or 'regression'.")

    # Create a Pandas Series of MI scores
    mi_scores = pd.Series(mi, index=dataset.columns)
    mi_scores = mi_scores.sort_values(ascending=False)

    # Plot Mutual Information Scores
    plt.figure(figsize=(10, 6))
    mi_scores.plot(kind='bar', color='skyblue')
    plt.title('Mutual Information Scores', fontsize=14)
    plt.ylabel('Mutual Information', fontsize=12)
    plt.xlabel('Features', fontsize=12)
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

    return mi_scores



drop_columns = ['latitude', 'longitude', 'no', 'nh3']

# For classification with dynamic bins
n_bins = 5
mi_scores = calculate_mutual_information(
    dataset=balanced_dataset.drop('aqi', axis=1),
    target=balanced_dataset['aqi'],
    task_type='classification',
    drop_columns=drop_columns,
    n_bins=n_bins  # Automatically adjusts bins
)

print(mi_scores)

In [ ]:
# MI with NO and NH3
drop_columns = ['latitude', 'longitude']

# For classification with dynamic bins
n_bins = 5
mi_scores = calculate_mutual_information(
    dataset=balanced_dataset.drop('aqi', axis=1),
    target=balanced_dataset['aqi'],
    task_type='classification',
    drop_columns=drop_columns,
    n_bins=n_bins  # Automatically adjusts bins
)

print(mi_scores)

### Result:
As we have seen Correlation and Covarience of NH3 and NO. They had poor relationship with AQI and the main factor was that they both have the same relation with CO and it showes some imbalance and by the way NO2 and CO and PM2.5 have can support NH3 and NO. In addition most sensors and countries do not count these two factors as AIR quality metric. But also they have their danger if their values are huge in air. So based on these we are going to drop these features, and lets see what will happen in the model.

In [ ]:
# Droping NO and NH3
droped_data = balanced_dataset.drop(['nh3', 'no'], axis=1)
droped_data

### Zero values:
Lets check zero values with ploting using scatter plot.

In [ ]:
# Visualize the zero values of each feature using a scatter plot
def zero_value(dataset):
    # Unwanted features to drop
    drop = ['longitude', 'latitude', 'aqi', 'year', 'month', 'day', 'hour', 'dayofweek']
    dataset = dataset.drop(drop, axis=1)

    # Iterate through each feature
    for feature in dataset.columns:
        # Get value counts and convert to a DataFrame
        value_counts = dataset[feature].value_counts().reset_index()
        value_counts.columns = [f"{feature}_value", "count"]

        # Plot using Plotly
        fig = px.scatter(
            value_counts,
            x=f"{feature}_value",
            y="count",
            title=f"Different values of {feature}",
            labels={f"{feature}_value": "Value", "count": "Count"}
        )
        fig.show()

    return


zero_value(droped_data)

### K-means Clustring:
We have different locations cordinates and we can not goes for all of them individually. So what we will do is using `K-mean cluster` algorithm to handle the cordinate and then use its ID.
* Finding Optimal cluster value.
  * Using elbow method
  * elbow and silhouette

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

def optimal_cluster(dataset):

  # Normalize latitude and longitude
  scaler = StandardScaler()
  dataset[['latitude', 'longitude']] = scaler.fit_transform(dataset[['latitude', 'longitude']])

  # Test k values
  distortions = []
  K = range(10, 101, 10)  # Test for 10 to 100 clusters
  for k in K:
      kmeans = KMeans(n_clusters=k, random_state=42)
      kmeans.fit(dataset[['latitude', 'longitude']])
      distortions.append(kmeans.inertia_)

  # Plot the Elbow Curve
  plt.plot(K, distortions, marker='o')
  plt.xlabel('Number of Clusters (k)')
  plt.ylabel('Distortion')
  plt.title('Elbow Method for Optimal Clusters')
  plt.show()
  return

optimal_cluster(droped_data)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

def optimal_cluster(dataset):
    # Normalize latitude and longitude
    scaler = StandardScaler()
    dataset[['latitude', 'longitude']] = scaler.fit_transform(dataset[['latitude', 'longitude']])

    # Elbow Method
    distortions = []
    K = range(10, 101, 10)  # Test for 10 to 100 clusters
    for k in K:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(dataset[['latitude', 'longitude']])
        distortions.append(kmeans.inertia_)

    # Plot the Elbow Curve
    plt.figure(figsize=(10, 6))
    plt.plot(K, distortions, marker='o', linestyle='--', color='b')
    plt.xlabel('Number of Clusters (k)')
    plt.ylabel('Distortion')
    plt.title('Elbow Method for Optimal Clusters')
    plt.grid(alpha=0.5)
    plt.show()

    # Silhouette Method
    silhouette_scores = []
    K_silhouette = range(10, 51, 5)  # Test for 10 to 50 clusters (for faster computation)
    for k in K_silhouette:
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(dataset[['latitude', 'longitude']])
        silhouette_scores.append(silhouette_score(dataset[['latitude', 'longitude']], labels))

    # Plot the Silhouette Scores
    plt.figure(figsize=(10, 6))
    plt.plot(K_silhouette, silhouette_scores, marker='o', linestyle='--', color='g')
    plt.xlabel('Number of Clusters (k)')
    plt.ylabel('Silhouette Score')
    plt.title('Silhouette Method for Optimal Clusters')
    plt.grid(alpha=0.5)
    plt.show()

    # Find optimal k from Silhouette Scores
    optimal_k = K_silhouette[silhouette_scores.index(max(silhouette_scores))]
    print(f"Optimal number of clusters based on Silhouette Method: {optimal_k}")

    return optimal_k


optimal_k = optimal_cluster(droped_data)
print(f"Optimal number of clusters: {optimal_k}")

### Result:
Based on the output of Elbow and Silhouette method and our desire of our project such that users location cordinate is important for out project to predict and classify well based on their location. and that is why we will go with `K = 50` or `Optimal cluster = 50`.
### Next Step:
Now it is time to we create our reagion_id feature based on k_means clustring using k value of 50.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pickle

# Step 1: Shuffle the dataset
def shuffle_dataset(dataset):
    """
    Shuffle the dataset to ensure data is mixed before clustering.
    """
    return dataset.sample(frac=1, random_state=42).reset_index(drop=True)

# Step 2: Define the function for region clustering
def add_region_id(dataset, k=50):
    """
    Add region_id feature to the dataset based on latitude and longitude clustering.
    """
    # Shuffle the dataset
    dataset = shuffle_dataset(dataset)

    # Normalize latitude and longitude
    scaler = StandardScaler()
    dataset[['latitude', 'longitude']] = scaler.fit_transform(dataset[['latitude', 'longitude']])
    with open("/content/drive/MyDrive/AFG-FTL-Capstone-Project/Models/cluster-scaler.sav", 'wb') as f:
        joblib.dump(scaler, f)

    # Apply KMeans clustering
    kmeans = KMeans(n_clusters=k, random_state=42)
    dataset['region_id'] = kmeans.fit_predict(dataset[['latitude', 'longitude']])

    # Save the KMeans model for future use
    with open('/content/drive/MyDrive/AFG-FTL-Capstone-Project/Models/kmeans_model.pkl', 'wb') as file:
      pickle.dump(kmeans, file)

    return dataset

# Step 3: Apply clustering to the dataset
cluster_data = add_region_id(droped_data)
cluster_data

In [ ]:
# Lets check count value of region_id
cluster_data['region_id'].value_counts()

In [ ]:
# Now lets drop latitude and longitude features
cluster_data = cluster_data.drop(['latitude', 'longitude'], axis=1)
cluster_data

### Result:
Perfect we have completed long journey. And we congertulate you.🎉🎉🎉🎉🎉<br>
Still you have to bare is for a small `EDA` Check up. to make sure everything smooths well.<br>
`Enjoy`😊😎😊😎😊😎


In [ ]:
# Simple EDA check up, to make sure of everything
# Null values
cluster_data.isna().sum()

In [ ]:
# Infinity value check up
np.array(np.isinf(cluster_data).sum())

In [ ]:
# Checking Duplicate
cluster_data.duplicated().sum()

In [ ]:
# Checking is dataframe
cluster_data[cluster_data.duplicated()]

In [ ]:
# Lets drop duplicated feature
cluster_data.drop_duplicates(inplace=True)

In [ ]:
# lets check again
cluster_data.duplicated().sum()

In [ ]:
# Function to visualize all data except 'aqi'
def all_data_vis(dataset):
    # Drop the 'aqi' column
    drop = ['aqi', 'year', 'month', 'day', 'hour', 'dayofweek', 'region_id']
    dataset = dataset.drop(drop, axis=1)

    # Melt the dataset to have all features in one column for easy plotting
    dataset_melted = dataset.melt(var_name='Feature', value_name='Value')

    # Create the scatter plot with Plotly Express
    fig = px.scatter(
        dataset_melted,
        x=dataset_melted.index,
        y='Value',
        color='Feature',
        title="Scatter Plot of Main features",
        labels={'Value': 'Feature Value', 'index': 'Index'},
        template='plotly_white'
    )

    fig.update_layout(
        title_font_size=20,
        legend_title_text='Feature',
        xaxis=dict(title='Index'),
        yaxis=dict(title='Value'),
        legend=dict(font=dict(size=10)),
    )

    fig.show()


all_data_vis(cluster_data)

In [ ]:
# Statistical analysis of data
cluster_data.describe()

In [ ]:
# checking full data report using profiling
from ydata_profiling import ProfileReport

profile = ProfileReport(cluster_data, title='Dataset Report')
profile

In [ ]:
# Saving the data
# cluster_data.to_csv("/content/drive/MyDrive/AFG-FTL-Capstone-Project/cluster_data.csv", index=False)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline

In [ ]:
cluster_data = pd.read_csv("/content/drive/MyDrive/AFG-FTL-Capstone-Project/cluster_data.csv")
cluster_data

### Balancing dataset based on AQI:
As you can see our dataset is not balanced based on AQI and this will riase biase on our model during training specially for classification models. So to handle them we need to balance our dataset.<br>
One best method for handling is SMOTEENN method. this method is more accurate then normal SMOTE method.

In [ ]:
from imblearn.combine import SMOTEENN
from collections import Counter

def balance(data):
    # Separate features and target
    X = data.drop('aqi', axis=1)  # Feature columns
    y = data['aqi']  # Target column

    # Check the class distribution before SMOTE
    print("Original class distribution:", Counter(y))

    # Apply SMOTE to balance the data
    smote_enn = SMOTEENN(random_state=42)
    X_resampled, y_resampled = smote_enn.fit_resample(X, y)

    # Check the class distribution after SMOTE
    print("Resampled class distribution:", Counter(y_resampled))

    # Combine resampled X and y into a new DataFrame (optional)
    balanced_data = pd.DataFrame(X_resampled, columns=X.columns)
    balanced_data['aqi'] = y_resampled
    return balanced_data

balanced_data = balance(cluster_data)
balanced_data

### Lag & Rolling:
For creating air quality prediction we need to lag and roll main features that based on these features and data the model be able to predict air quality for 1,3,6,24,48 and 72 hours later or future.

In [ ]:
"""
  Lag and Rolling is used to create trend and seasonality on dataset for regression model
  and as we do not have much amount of data and specially historical data we need to handle it
  using roll and lag and create features for prediction of specific time perioud.
"""
def create_lag_and_rolling_features(df, num_cols, time_cols, lags, rolling_windows):
    # Initialize DataFrames for lagged and rolling features
    lagged_features = pd.DataFrame()
    rolling_features = pd.DataFrame()

    # Create lag and rolling features for each numeric column
    for col in num_cols:
        # Lag features
        for lag in lags:
            lagged_features[f"{col}_lag_{lag}"] = df[col].shift(lag)

        # Rolling features
        for window in rolling_windows:
            rolling_features[f"{col}_roll_mean_{window}"] = df[col].rolling(window=window).mean()
            rolling_features[f"{col}_roll_std_{window}"] = df[col].rolling(window=window).std()

    # Combine lag and rolling features
    combined_features = pd.concat([lagged_features, rolling_features], axis=1)

    # Combine with original time features
    final_df = pd.concat([df[time_cols + num_cols], combined_features], axis=1)

    # Fill missing values from lag and rolling operations using forward fill
    final_df.fillna(method='ffill', inplace=True)

    # Drop remaining NaNs (e.g., at the start of the dataset where no lag/rolling data exists)
    final_df.dropna(inplace=True)

    # Reset index for a clean dataset
    final_df.reset_index(drop=True, inplace=True)

    return final_df


# Parameters
num_cols = ['co', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'aqi']
time_cols = ['year', 'month', 'day', 'hour', 'dayofweek', 'region_id']
lags = [1, 3, 6, 24, 48, 72]
rolling_windows = [3, 6, 24]

# Call the function
processed_data = create_lag_and_rolling_features(balanced_data, num_cols, time_cols, lags, rolling_windows)

processed_data.head()

In [ ]:
processed_data.columns

In [ ]:
for row, col in processed_data.isna().sum().items():
  print(row, col)

In [ ]:
# checking after lag and rolling some features to be in its type
for x in ['year', 'month', 'day', 'hour', 'dayofweek', 'aqi']:
  print(processed_data[x].value_counts())

In [ ]:
# Statistical analysis of new data
processed_data.describe()

### Missing values After Creating Lag and Rolling.
We are going to fill using KNN and lag based imputation.

In [ ]:
from sklearn.impute import KNNImputer

# 1. Define a function to apply KNN imputation to columns dynamically
def apply_knn_imputation(df, threshold=0.3):
    knn_columns = df.columns[df.isna().mean() < threshold]  # Columns with <30% missing values
    knn_data = df[knn_columns]

    # Initialize KNN imputer
    knn_imputer = KNNImputer(n_neighbors=5)

    # Apply KNN imputation
    knn_imputed_data = pd.DataFrame(knn_imputer.fit_transform(knn_data), columns=knn_columns)

    # Replace the original columns with the KNN imputed ones
    df[knn_columns] = knn_imputed_data
    return df

# 2. Define a function to apply lag-based imputation for time-dependent features
def lag_imputation(df, threshold=0.3, lag_hours=1):
    lag_columns = [col for col in df.columns if "lag" in col and df[col].isna().mean() > threshold]
    for col in lag_columns:
        df[col] = df[col].fillna(df[col].shift(lag_hours))  # Use previous time step for imputation
    return df

# 3. Define a function to apply rolling mean imputation
def rolling_imputation(df, threshold=0.3, window=3):
    rolling_columns = [col for col in df.columns if "rolling" in col and df[col].isna().mean() > threshold]
    for col in rolling_columns:
        df[col] = df[col].fillna(df[col].rolling(window=window, min_periods=1).mean())  # Rolling mean
    return df

# Apply KNN imputation (for columns with less than 30% missing data)
data = apply_knn_imputation(processed_data)

# Apply Lag-based imputation (for columns with lag in the name and >30% missing)
data = lag_imputation(data)

# Apply Rolling mean imputation (for columns with rolling in the name and >30% missing)
data = rolling_imputation(data)


assert data.isna().sum().sum() == 0, "There are still missing values in the dataset."


In [ ]:
data.isna().sum().sum()

In [ ]:
# checking after lag and rolling some features to be in its type
for x in ['year', 'month', 'day', 'hour', 'dayofweek', 'aqi']:
  print(data[x].value_counts())

In [ ]:
# data.to_csv("/content/drive/MyDrive/AFG-FTL-Capstone-Project/processed_data.csv", index=False)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline

In [ ]:
# read data
data = pd.read_csv("added the cleaned dataset")
data

In [ ]:
# ploting the regression of time series data
for feature in data.columns:
  if feature in ['aqi_lag_24', 'aqi_lag_48', 'aqi_lag_72', 'aqi_lag_1', 'aqi_lag_3', 'aqi_lag_6']:
    plt.plot(data[feature])
    plt.show()

### Ready Time to select and train Our Model 😉😎
But Before Selecting our model we are going to encode the target variables for classification and spliting the data.<br>
In this some cells we are going to train 3 different model.
* **CNN**
* **LSTM**
* **Transformer**

Time to Enjoy.🎉😎😎😎😎😎😎😎

In [ ]:
data.describe()

### Encoding.
Using OneHotEncoder to encode the aqi.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

def encode_classification_targets(classification_targets):
    """
    Encodes classification targets using one-hot encoding.

    Parameters:
        classification_targets (Series or ndarray): Classification target labels.

    Returns:
        Tensor: One-hot encoded classification targets.
    """
    encoder = OneHotEncoder(sparse_output=False)
    encoded_targets = encoder.fit_transform(classification_targets.values.reshape(-1, 1))
    return torch.tensor(encoded_targets, dtype=torch.float32), encoder


### Spliting Data:
Splite data into train, val and test.

In [ ]:
def convert_to_tensors(features, regression_targets):
    """
    Converts features and regression targets to PyTorch tensors.

    Returns:
        Tuple: PyTorch tensors for features and regression targets.
    """
    features_tensor = torch.tensor(features.values, dtype=torch.float32)
    regression_tensor = torch.tensor(regression_targets.values, dtype=torch.float32)
    return features_tensor, regression_tensor

In [ ]:
import torch
from sklearn.model_selection import train_test_split

def split_data(features, regression_targets, classification_features, classification_targets,
               test_size=0.2, random_state=42):
    """
    Splits the data into training and testing sets for both regression and classification tasks.

    Parameters:
        features (Tensor): Features tensor for regression.
        regression_targets (Tensor): Regression target tensor.
        classification_features (Tensor): Main features for classification.
        classification_targets (Tensor): Encoded classification targets.
        test_size (float): Proportion of the data to use for testing.
        random_state (int): Random state for reproducibility.

    Returns:
        Tuple: Training and testing sets for regression and classification tasks.
    """
    # Regression split
    X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
        features, regression_targets, test_size=test_size, random_state=random_state
    )

    # Classification split
    X_class_train, X_class_test, y_class_train, y_class_test = train_test_split(
        classification_features, classification_targets, test_size=test_size, random_state=random_state
    )

    return (X_reg_train, X_reg_test, y_reg_train, y_reg_test,
            X_class_train, X_class_test, y_class_train, y_class_test)

In [ ]:
def select_classification_features(data, feature_columns):
    """
    Selects specific features for classification tasks.

    Parameters:
        data (DataFrame): Input data.
        feature_columns (list): List of column names to select for classification.

    Returns:
        DataFrame: Selected features for classification.
    """
    return data[feature_columns]


In [ ]:
# Regression features and targets (unchanged)
features = data[['aqi_lag_24', 'aqi_lag_48', 'aqi_lag_72', 'aqi_lag_1', 'aqi_lag_3', 'aqi_lag_6']]
regression_targets = data[['aqi_shift_24h', 'aqi_shift_48h', 'aqi_shift_72h']]

# Classification features (main features) and target
main_features = ['co', 'no2', 'o3', 'so2', 'pm2_5', 'pm10']
classification_features = select_classification_features(data, main_features)
classification_targets = data['aqi']

# Encode classification targets
classification_tensor, encoder = encode_classification_targets(classification_targets)

# Convert features and regression targets to tensors
features_tensor, regression_tensor = convert_to_tensors(features, regression_targets)
classification_features_tensor = torch.tensor(classification_features.values, dtype=torch.float32)

# Split data
(X_reg_train, X_reg_test, y_reg_train, y_reg_test,
 X_class_train, X_class_test, y_class_train, y_class_test) = split_data(
    features_tensor, regression_tensor, classification_features_tensor, classification_tensor,
    test_size=0.2, random_state=42
)

### Finally time to Train our First Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


# Assuming input tensors are already defined:
# X_reg_train, y_reg_train, y_class_train, X_reg_test, y_reg_test, y_class_test

# Define the Multi-Task CNN Model
class MultiTaskAQIModel(nn.Module):
    def __init__(self, input_size, num_classes, output_size):
        super(MultiTaskAQIModel, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(16)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(32)
        self.shared_fc = nn.Linear(32 * input_size, 128)

        # Regression head
        self.reg_fc = nn.Linear(128, output_size)

        # Classification head
        self.class_fc = nn.Linear(128, num_classes)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension for Conv1D
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.shared_fc(x))
        x = self.dropout(x)

        # Separate outputs
        regression_output = self.reg_fc(x)
        classification_output = self.class_fc(x)

        return regression_output, classification_output

# Training Function
def train_multi_task_model(X_train, y_reg_train, y_class_train, X_test, y_reg_test, y_class_test,
                           input_size, num_classes, output_size, epochs, batch_size, learning_rate, save_model_path):
    # Convert to PyTorch DataLoader
    train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                                   torch.tensor(y_reg_train, dtype=torch.float32),
                                   torch.tensor(y_class_train, dtype=torch.float32))
    test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                                  torch.tensor(y_reg_test, dtype=torch.float32),
                                  torch.tensor(y_class_test, dtype=torch.float32))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model, optimizer, and loss functions
    model = MultiTaskAQIModel(input_size=input_size, num_classes=num_classes, output_size=output_size)
    regression_criterion = nn.MSELoss()
    classification_criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training Loop
    train_history = {'reg_loss': [], 'class_loss': []}
    for epoch in range(epochs):
        model.train()
        train_reg_loss, train_class_loss = 0.0, 0.0

        for X_batch, y_reg_batch, y_class_batch in train_loader:
            optimizer.zero_grad()
            reg_output, class_output = model(X_batch)

            # Compute losses
            reg_loss = regression_criterion(reg_output, y_reg_batch)
            class_loss = classification_criterion(class_output, y_class_batch.argmax(dim=1))
            loss = reg_loss + class_loss

            loss.backward()
            optimizer.step()

            train_reg_loss += reg_loss.item()
            train_class_loss += class_loss.item()

        # Record losses
        train_history['reg_loss'].append(train_reg_loss / len(train_loader))
        train_history['class_loss'].append(train_class_loss / len(train_loader))
        print(f"Epoch {epoch + 1}/{epochs}, Regression Loss: {train_history['reg_loss'][-1]}, Classification Loss: {train_history['class_loss'][-1]}")

    # Save the trained model
    torch.save(model.state_dict(), save_model_path)
    print(f"Model saved at {save_model_path}!")

    # Return results
    return {
        'model': model,
        'train_loader': train_loader,
        'test_loader': test_loader,
        'train_history': train_history
    }

# Variable Declarations
input_size = X_reg_train.shape[1]  # Number of features in regression input
num_classes = y_class_train.shape[1]  # Number of classes for classification
output_size = y_reg_train.shape[1]  # Number of regression outputs

# Train the model
results = train_multi_task_model(
    X_train=X_reg_train, y_reg_train=y_reg_train, y_class_train=y_class_train,
    X_test=X_reg_test, y_reg_test=y_reg_test, y_class_test=y_class_test,
    input_size=input_size, num_classes=num_classes, output_size=output_size,
    epochs=10, batch_size=64, learning_rate=0.001,
    save_model_path="/content/drive/MyDrive/AFG-FTL-Capstone-Project/Models/multi_task_aqi_model.pth"
)

# Extract variables for evaluation and visualization
trained_model = results['model']
train_loader = results['train_loader']
test_loader = results['test_loader']
train_history = results['train_history']


In [ ]:
# Plot the losses over epochs
plt.figure(figsize=(10, 5))
plt.plot(train_history['reg_loss'], label="Regression Loss")
plt.plot(train_history['class_loss'], label="Classification Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training Loss per Epoch")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error, mean_absolute_error, r2_score
import numpy as np

def evaluate_model(model, test_loader):
    model.eval()  # Set model to evaluation mode
    y_reg_true, y_reg_pred = [], []
    y_class_true, y_class_pred = [], []

    with torch.no_grad():
        for X_batch, y_reg_batch, y_class_batch in test_loader:
            reg_output, class_output = model(X_batch)

            # Regression predictions
            y_reg_true.append(y_reg_batch.numpy())
            y_reg_pred.append(reg_output.numpy())

            # Classification predictions
            y_class_true.append(y_class_batch.numpy())
            y_class_pred.append(class_output.numpy())

    # Concatenate all batches
    y_reg_true = np.concatenate(y_reg_true, axis=0)
    y_reg_pred = np.concatenate(y_reg_pred, axis=0)
    y_class_true = np.argmax(np.concatenate(y_class_true, axis=0), axis=1)  # One-hot to class index
    y_class_pred = np.argmax(np.concatenate(y_class_pred, axis=0), axis=1)  # Class probabilities to index

    # Classification Metrics
    classification_acc = accuracy_score(y_class_true, y_class_pred)
    classification_report_str = classification_report(y_class_true, y_class_pred)

    # Regression Metrics
    mse = mean_squared_error(y_reg_true, y_reg_pred)
    mae = mean_absolute_error(y_reg_true, y_reg_pred)
    r2 = r2_score(y_reg_true, y_reg_pred)

    # Print Metrics
    print("Classification Metrics:")
    print(f"Accuracy: {classification_acc:.4f}")
    print("\nClassification Report:")
    print(classification_report_str)

    print("\nRegression Metrics:")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"R-Squared (R²): {r2:.4f}")

    # Return metrics if needed for further processing
    return {
        'classification': {
            'accuracy': classification_acc,
            'report': classification_report_str,
        },
        'regression': {
            'mse': mse,
            'mae': mae,
            'r2': r2,
        }
    }

# Evaluate the trained model on the test data
metrics = evaluate_model(trained_model, test_loader)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


features = data[['aqi_shift_24h', 'aqi_shift_48h', 'aqi_shift_72h', 'aqi_lag_1h', 'aqi_lag_3h', 'aqi_lag_6h']]
regression_targets = data[['aqi_shift_24h', 'aqi_shift_48h', 'aqi_shift_72h']]
classification_targets = data['aqi']  # AQI classification (1 to 5)

# One-hot encode classification targets
encoder = OneHotEncoder(sparse_output=False)
classification_targets = encoder.fit_transform(classification_targets.values.reshape(-1, 1))

# Convert to PyTorch tensors
features_tensor = torch.tensor(features.values, dtype=torch.float32)
regression_tensor = torch.tensor(regression_targets.values, dtype=torch.float32)
classification_tensor = torch.tensor(classification_targets, dtype=torch.float32)

# Train-test split
X_train, X_test, y_reg_train, y_reg_test, y_class_train, y_class_test = train_test_split(
    features_tensor, regression_tensor, classification_tensor, test_size=0.2, random_state=42
)

# Convert to PyTorch DataLoader
batch_size = 64

train_dataset = torch.utils.data.TensorDataset(X_train, y_reg_train, y_class_train)
test_dataset = torch.utils.data.TensorDataset(X_test, y_reg_test, y_class_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the Multi-Task CNN Model
class MultiTaskAQIModel(nn.Module):
    def __init__(self, input_size, num_classes, output_size):
        super(MultiTaskAQIModel, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(16)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(32)
        self.shared_fc = nn.Linear(32 * input_size, 128)

        # Regression head
        self.reg_fc = nn.Linear(128, output_size)

        # Classification head
        self.class_fc = nn.Linear(128, num_classes)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension for Conv1D
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.shared_fc(x))
        x = self.dropout(x)

        # Separate outputs
        regression_output = self.reg_fc(x)
        classification_output = self.class_fc(x)

        return regression_output, classification_output

# Initialize the model, optimizer, and loss functions
input_size = features_tensor.shape[1]
num_classes = classification_targets.shape[1]
output_size = regression_targets.shape[1]

model = MultiTaskAQIModel(input_size=input_size, num_classes=num_classes, output_size=output_size)

regression_criterion = nn.MSELoss()
classification_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 10
for epoch in range(epochs):
    model.train()
    train_reg_loss, train_class_loss = 0.0, 0.0
    for X_batch, y_reg_batch, y_class_batch in train_loader:
        optimizer.zero_grad()
        reg_output, class_output = model(X_batch)

        # Compute losses
        reg_loss = regression_criterion(reg_output, y_reg_batch)
        class_loss = classification_criterion(class_output, y_class_batch.argmax(dim=1))

        loss = reg_loss + class_loss  # Combined loss
        loss.backward()
        optimizer.step()

        train_reg_loss += reg_loss.item()
        train_class_loss += class_loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Regression Loss: {train_reg_loss / len(train_loader)}, Classification Loss: {train_class_loss / len(train_loader)}")

# Save the model
# torch.save(model.state_dict(), "/content/drive/MyDrive/AFG-FTL-Capstone-Project/Models/multi_task_aqi_model.pth")
# print("Model saved!")

# Evaluation
model.eval()
reg_preds, class_preds, reg_true, class_true = [], [], [], []

with torch.no_grad():
    for X_batch, y_reg_batch, y_class_batch in test_loader:
        reg_output, class_output = model(X_batch)
        reg_preds.append(reg_output)
        class_preds.append(class_output)
        reg_true.append(y_reg_batch)
        class_true.append(y_class_batch)

reg_preds = torch.cat(reg_preds).numpy()
class_preds = torch.cat(class_preds).argmax(dim=1).numpy()
reg_true = torch.cat(reg_true).numpy()
class_true = torch.cat(class_true).argmax(dim=1).numpy()

# Calculate metrics
from sklearn.metrics import mean_squared_error, accuracy_score, r2_score

# Regression metrics
for i, horizon in enumerate(['24h', '48h', '72h']):
    rmse = np.sqrt(mean_squared_error(reg_true[:, i], reg_preds[:, i]))
    r2 = r2_score(reg_true[:, i], reg_preds[:, i])
    print(f"{horizon} Regression - RMSE: {rmse:.2f}, R²: {r2:.2f}")

# Classification accuracy
accuracy = accuracy_score(class_true, class_preds)
print(f"Classification Accuracy: {accuracy:.2f}")


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 1. Standardize the features
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test.numpy())

# 2. Apply PCA for dimensionality reduction to 2D
pca = PCA(n_components=2)
X_test_pca = pca.fit_transform(X_test_scaled)

# 3. Create a meshgrid in the 2D PCA-transformed feature space
x_min, x_max = X_test_pca[:, 0].min() - 1, X_test_pca[:, 0].max() + 1
y_min, y_max = X_test_pca[:, 1].min() - 1, X_test_pca[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1))

# 4. Map the meshgrid back to the original feature space
grid_points_pca = np.c_[xx.ravel(), yy.ravel()]
grid_points_original = pca.inverse_transform(grid_points_pca)
grid_points_tensor = torch.tensor(scaler.inverse_transform(grid_points_original), dtype=torch.float32)

# 5. Predict the classification output for each point in the meshgrid
model.eval()
with torch.no_grad():
    _, grid_preds = model(grid_points_tensor)
grid_preds = grid_preds.argmax(dim=1).numpy().reshape(xx.shape)

# 6. Plot the decision boundary
plt.figure(figsize=(10, 6))
plt.contourf(xx, yy, grid_preds, alpha=0.8, cmap=plt.cm.Paired)
plt.scatter(X_test_pca[:, 0], X_test_pca[:, 1], c=class_true, edgecolor='k', cmap=plt.cm.Paired)
plt.title("Decision Boundary for Classification")
plt.xlabel("PCA Feature 1")
plt.ylabel("PCA Feature 2")
plt.colorbar(label="Predicted Class")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler

# Step 1: Preprocess the Data
X = data[['co', 'no2', 'o3', 'so2', 'pm2_5', 'pm10']]
y = data['aqi']
# Step 2: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 3: Train SVM
svm_model = SVC(kernel='rbf', random_state=42)  # Radial basis function kernel
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)

# Step 4: Train Random Forest
rf_model = RandomForestClassifier(random_state=42, n_estimators=100)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

# Step 5: Evaluate Models
print("SVM Classification Report:")
print(classification_report(y_test, svm_predictions))
print("SVM Accuracy:", accuracy_score(y_test, svm_predictions))

print("\nRandom Forest Classification Report:")
print(classification_report(y_test, rf_predictions))
print("Random Forest Accuracy:", accuracy_score(y_test, rf_predictions))


### Checking Classification on different models.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/AFG-FTL-Capstone-Project/processed_data.csv")
data

### Testing different Activation function on CNN

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Function to encode ProtocolName using one-hot encoding and rename the feature to Label
def encode_protocol_name(dataset):
    # Initialize the OneHotEncoder
    encoder = OneHotEncoder(sparse_output=False)

    # Extract the 'ProtocolName' column and reshape it to 2D array
    protocol_name_column = dataset[['aqi']]

    # Fit and transform the 'ProtocolName' column
    one_hot_encoded = encoder.fit_transform(protocol_name_column)

    # Convert the one-hot encoded array into a DataFrame
    one_hot_encoded_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(['aqi']))

    # Drop the original ProtocolName column
    dataset.drop('aqi', axis=1, inplace=True)

    # Concatenate the original dataset with the one-hot encoded DataFrame
    dataset = pd.concat([data, one_hot_encoded_df], axis=1)

    return dataset

new_data = encode_protocol_name(data)
new_data

In [ ]:
# importing train_test_split
from sklearn.model_selection import train_test_split

# split function
def split_data(dataset):
    dataset = dataset.sample(frac=1, random_state=1).reset_index(drop=True)

    one_hot_columns = [col for col in dataset.columns if col in ['aqi_1.0',	'aqi_2.0',	'aqi_3.0',	'aqi_4.0',	'aqi_5.0']]

    # Split the dataset into X (features) and y (labels)
    x = dataset[['co', 'no2', 'o3', 'so2', 'pm2_5', 'pm10']]
    y = dataset[one_hot_columns]

    # Split the data into train, validation, and test sets
    x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.3, random_state=1)
    x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.7, random_state=1)



    return x_train, x_val, x_test, y_train, y_val, y_test

In [ ]:
# convert Pandas DataFrame into Tensorflow Tensor
import tensorflow as tf
def dataframe_to_tensor():
  '''
  This function is going to convert Pandas DataFrame into TensorFlow Tensor
  and then Reshape them from 2D to 3D for model fitting.
  And at last change them to tensorflow dataset and shuffle them for better model operation.
  '''
  # set random seed
  tf.random.set_seed(1)

  # train validate and test the dataset first using scikit-learn then convert them into tensor
  x_train, x_val, x_test, y_train, y_val, y_test = split_data(new_data)

  # DataFrame to TensorFlow Tensor
  x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
  x_val = tf.convert_to_tensor(x_val, dtype=tf.float32)
  x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)
  y_train = tf.convert_to_tensor(y_train, dtype=tf.int16)
  y_val = tf.convert_to_tensor(y_val, dtype=tf.int16)
  y_test = tf.convert_to_tensor(y_test, dtype=tf.int16)


  # reshape the tensor from 2D to 3D
  x_train = tf.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
  x_val = tf.reshape(x_val, (x_val.shape[0], x_val.shape[1], 1))
  x_test = tf.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

  # Convert to tf.data.Dataset
  train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
  val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
  test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

  # Shuffle and batch the datasets
  batch_size = 32  # Try reducing this further if needed
  train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
  val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
  test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)


  return train_dataset, val_dataset, test_dataset, x_test, y_test

In [ ]:
train_dataset, val_dataset, test_dataset, x_test, y_test = dataframe_to_tensor()

In [ ]:
# CNN model
from tensorflow.keras import layers, models
import time
import pickle

def create_model(input_shape, num_classes):
    model = models.Sequential()

    # Add Conv1D layers with padding to avoid negative dimensions
    model.add(layers.Conv1D(32, 4, padding='same', activation='tanh', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D(2))
    # model.add(layers.Dropout(0.3))

    # model.add(layers.Conv1D(64, 4, padding='same', activation='gelu'))
    # model.add(layers.BatchNormalization())
    # model.add(layers.MaxPooling1D(2))
    # # model.add(layers.Dropout(0.3))


    # Flatten the output and add Dense layers for classification
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='tanh'))
    model.add(layers.BatchNormalization())
    # model.add(layers.Dropout(0.4))
    model.add(layers.Dense(num_classes, activation='softmax'))  # Multi-class classification, use softmax activation

    return model

# Example usage:
input_shape = (6, 1)  # Update this to match your dataset's input shape
num_classes = 5       # Update this to match the number of classes in your dataset

model = create_model(input_shape, num_classes)

# Compile the model

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',  # Use sparse_categorical_crossentropy if labels are not one-hot encoded
              metrics=['accuracy'])

# Summary of the model
model.summary()

# Assume train_dataset, val_dataset, and test_dataset are already created and preprocessed
# timeout
start = time.time()

# Train the model with callbacks
history = model.fit(train_dataset,
                    epochs=15,
                    validation_data=val_dataset)

end = time.time()

# Save the model
# model.save('/content/drive/MyDrive/Final-Project/Saved-models/AppMultiClassification/final_multiclass.h5')

# # Save the training history
# with open('/content/drive/MyDrive/Final-Project/Saved-models/AppMultiClassification/final_multiclass.pkl', 'wb') as file:
#     pickle.dump(history.history, file)

duration = end - start

print(f'Training time: {duration:.2f} seconds')

# Evaluate the model on the test dataset
test_loss, test_acc = model.evaluate(test_dataset)
print('Test accuracy:', test_acc)

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# 1. Standardize the features
scaler = StandardScaler()
x_test_scaled = scaler.fit_transform(x_test.numpy().reshape(-1, 6))  # Flatten input for scaling

# 2. Apply PCA for dimensionality reduction to 2D
pca = PCA(n_components=2)
x_test_pca = pca.fit_transform(x_test_scaled)

# 3. Create a meshgrid in the 2D PCA-transformed feature space
x_min, x_max = x_test_pca[:, 0].min() - 1, x_test_pca[:, 0].max() + 1
y_min, y_max = x_test_pca[:, 1].min() - 1, x_test_pca[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1))

# 4. Map the meshgrid back to the original feature space
grid_points_pca = np.c_[xx.ravel(), yy.ravel()]
grid_points_original = pca.inverse_transform(grid_points_pca)
grid_points_scaled = scaler.inverse_transform(grid_points_original)
grid_points_tensor = tf.convert_to_tensor(grid_points_scaled, dtype=tf.float32)

# 5. Predict the classification output for each point in the meshgrid
model.evaluate(x_test, y_test)  # Optional evaluation step
grid_preds = model.predict(grid_points_tensor)
grid_preds = np.argmax(grid_preds, axis=1).reshape(xx.shape)

# 6. Plot the decision boundary
plt.figure(figsize=(10, 6))
plt.contourf(xx, yy, grid_preds, alpha=0.8, cmap=plt.cm.Paired)

# Ensure class_true is properly defined
if len(y_test.shape) > 1 and y_test.shape[1] > 1:
    class_true = np.argmax(y_test, axis=1)  # Convert one-hot to class indices
else:
    class_true = y_test.reshape(-1)

plt.scatter(x_test_pca[:, 0], x_test_pca[:, 1], c=class_true, edgecolor='k', cmap=plt.cm.Paired)
plt.title("Decision Boundary for Classification")
plt.xlabel("PCA Feature 1")
plt.ylabel("PCA Feature 2")
plt.colorbar(label="Predicted Class")
plt.show()


In [ ]:
# CNN model
from tensorflow.keras import layers, models
import time
import pickle

def create_model(input_shape, num_classes):
    model = models.Sequential()

    # Add Conv1D layers with padding to avoid negative dimensions
    model.add(layers.Conv1D(32, 4, padding='same', activation='tanh', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D(2))
    # model.add(layers.Dropout(0.3))

    model.add(layers.Conv1D(64, 4, padding='same', activation='gelu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D(2))
    # model.add(layers.Dropout(0.3))


    # Flatten the output and add Dense layers for classification
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='tanh'))
    model.add(layers.BatchNormalization())
    # model.add(layers.Dropout(0.4))
    model.add(layers.Dense(num_classes, activation='softmax'))  # Multi-class classification, use softmax activation

    return model

# Example usage:
input_shape = (12, 1)  # Update this to match your dataset's input shape
num_classes = 5       # Update this to match the number of classes in your dataset

model = create_model(input_shape, num_classes)

# Compile the model

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',  # Use sparse_categorical_crossentropy if labels are not one-hot encoded
              metrics=['accuracy'])

# Summary of the model
model.summary()

# Assume train_dataset, val_dataset, and test_dataset are already created and preprocessed
# timeout
start = time.time()

# Train the model with callbacks
history = model.fit(train_dataset,
                    epochs=15,
                    validation_data=val_dataset)

end = time.time()

# Save the model
# model.save('/content/drive/MyDrive/Final-Project/Saved-models/AppMultiClassification/final_multiclass.h5')

# # Save the training history
# with open('/content/drive/MyDrive/Final-Project/Saved-models/AppMultiClassification/final_multiclass.pkl', 'wb') as file:
#     pickle.dump(history.history, file)

duration = end - start

print(f'Training time: {duration:.2f} seconds')

# Evaluate the model on the test dataset
test_loss, test_acc = model.evaluate(test_dataset)
print('Test accuracy:', test_acc)

In [ ]:
# Hyperprameter tunning
import tensorflow as tf
from tensorflow.keras import layers, models
from skopt import gp_minimize
from skopt.space import Integer, Real, Categorical
from skopt.utils import use_named_args
import numpy as np

# Define the search space for hyperparameters
search_space = [
    Integer(16, 128, name='conv1_filters'),  # Number of filters in the first Conv1D layer
    Integer(16, 128, name='conv2_filters'),  # Number of filters in the second Conv1D layer
    Integer(32, 256, name='dense_units'),    # Number of units in Dense layer
    Real(1e-4, 1e-2, prior='log-uniform', name='learning_rate'),  # Learning rate
    Categorical(['tanh', 'relu'], name='activation'),  # Activation function
]

# Objective function for Bayesian Optimization
@use_named_args(search_space)
def objective_function(**params):
    # Build the CNN model
    model = models.Sequential()
    model.add(layers.Conv1D(params['conv1_filters'], kernel_size=4, activation=params['activation'], padding='same', input_shape=(6, 1)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D(2))

    model.add(layers.Conv1D(params['conv2_filters'], kernel_size=4, activation=params['activation'], padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D(2))

    model.add(layers.Flatten())
    model.add(layers.Dense(params['dense_units'], activation=params['activation']))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(5, activation='softmax'))  # Assume 5 classes for multi-class classification

    # Compile the model
    optimizer = tf.optimizers.Adam(learning_rate=params['learning_rate'])
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model on a subset of data
    history = model.fit(train_dataset, epochs=5, validation_data=val_dataset, verbose=0)  # Use fewer epochs for faster tuning

    # Return validation accuracy (objective to maximize)
    val_accuracy = max(history.history['val_accuracy'])
    return -val_accuracy  # Negative because gp_minimize minimizes the objective function

# Run Bayesian Optimization
result = gp_minimize(objective_function, search_space, n_calls=20, random_state=42)

# Print the best parameters
print("Best hyperparameters:")
print(f"Conv1 Filters: {result.x[0]}")
print(f"Conv2 Filters: {result.x[1]}")
print(f"Dense Units: {result.x[2]}")
print(f"Learning Rate: {result.x[3]}")
print(f"Activation: {result.x[4]}")

# Train the final model with the best parameters
best_params = {
    'conv1_filters': result.x[0],
    'conv2_filters': result.x[1],
    'dense_units': result.x[2],
    'learning_rate': result.x[3],
    'activation': result.x[4]
}

final_model = models.Sequential()
final_model.add(layers.Conv1D(best_params['conv1_filters'], kernel_size=4, activation=best_params['activation'], padding='same', input_shape=(6, 1)))
final_model.add(layers.BatchNormalization())
final_model.add(layers.MaxPooling1D(2))

final_model.add(layers.Conv1D(best_params['conv2_filters'], kernel_size=4, activation=best_params['activation'], padding='same'))
final_model.add(layers.BatchNormalization())
final_model.add(layers.MaxPooling1D(2))

final_model.add(layers.Flatten())
final_model.add(layers.Dense(best_params['dense_units'], activation=best_params['activation']))
final_model.add(layers.BatchNormalization())
final_model.add(layers.Dense(5, activation='softmax'))

optimizer = tf.optimizers.Adam(learning_rate=best_params['learning_rate'])
final_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the final model
history = final_model.fit(train_dataset, epochs=15, validation_data=val_dataset)

# Evaluate the model
test_loss, test_acc = final_model.evaluate(test_dataset)
print("Test accuracy with optimized hyperparameters:", test_acc)

In [ ]:
# Best Hyperparameter after tunning lets find out
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the optimized CNN model
def create_optimized_model(input_shape, num_classes):
    model = models.Sequential()

    # First Conv1D layer with optimized number of filters and activation function
    model.add(layers.Conv1D(105, kernel_size=4, activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D(pool_size=2))

    # Second Conv1D layer
    model.add(layers.Conv1D(58, kernel_size=4, activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D(pool_size=2))

    # Flatten the output and add Dense layers
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(num_classes, activation='softmax'))  # Multi-class classification

    return model

# Define input shape and number of classes
input_shape = (6, 1)  # Update to match your dataset
num_classes = 5       # Update to match your dataset

# Create the model
model = create_optimized_model(input_shape, num_classes)

# Compile the model with the optimized learning rate
optimizer = tf.optimizers.Adam(learning_rate=0.00041633970544941064)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

# Assume train_dataset, val_dataset, and test_dataset are already prepared
# Train the model
history = model.fit(train_dataset,
                    epochs=15,
                    validation_data=val_dataset)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_dataset)
print("Test accuracy with optimized hyperparameters:", test_acc)


In [ ]:
# Adding two more convolutional layer and lets check
from tensorflow.keras import layers, models
import time
import pickle

def create_model(input_shape, num_classes):
    model = models.Sequential()

    # First Conv1D layer
    model.add(layers.Conv1D(32, 4, padding='same', activation='tanh', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D(2))

    # Second Conv1D layer
    model.add(layers.Conv1D(64, 4, padding='same', activation='gelu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D(2))

    # Third Conv1D layer
    model.add(layers.Conv1D(128, 4, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D(2))  # Use pooling here if the spatial dimensions allow.

    # Fourth Conv1D layer
    model.add(layers.Conv1D(256, 4, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())

    # Use Global Pooling instead of MaxPooling to avoid spatial collapse
    model.add(layers.GlobalMaxPooling1D())

    # Flatten the output and add Dense layers for classification
    model.add(layers.Dense(64, activation='tanh'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(num_classes, activation='softmax'))  # Multi-class classification

    return model

# Example usage:
input_shape = (12, 1)  # Ensure this matches your dataset's input shape
num_classes = 5        # Update this to match the number of classes in your dataset

model = create_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Summary of the model
model.summary()

# Assume train_dataset, val_dataset, and test_dataset are already created and preprocessed
# Measure training time
start = time.time()

# Train the model
history = model.fit(train_dataset,
                    epochs=15,
                    validation_data=val_dataset)

end = time.time()

# Save the model and training history (optional)
# model.save('/content/drive/MyDrive/Final-Project/Saved-models/AppMultiClassification/final_multiclass.h5')
# with open('/content/drive/MyDrive/Final-Project/Saved-models/AppMultiClassification/final_multiclass.pkl', 'wb') as file:
#     pickle.dump(history.history, file)

duration = end - start
print(f'Training time: {duration:.2f} seconds')

# Evaluate the model on the test dataset
test_loss, test_acc = model.evaluate(test_dataset)
print('Test accuracy:', test_acc)

In [ ]:
# new lastm model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Model architecture
model = Sequential([
    LSTM(64, input_shape=((12,1)), return_sequences=True),
    Dropout(0.3),
    LSTM(32, return_sequences=False),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(5, activation='softmax')  # 5 classes in the target variable
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_dataset,validation_data=val_dataset, epochs=10, batch_size=32, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")


In [ ]:
# LSTM algorithm
from tensorflow.keras import layers, models
def create_rnn_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.LSTM(64, return_sequences=True, input_shape=input_shape))
    model.add(layers.LSTM(64))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

# Example usage:
input_shape = (6, 1)  # dataset's input shape
num_classes = 5       # Number of classes in dataset
model = create_rnn_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# train the model
history = model.fit(train_dataset, validation_data=val_dataset, epochs=8)
# model.save("/kaggle/working/rnn_multi_app.h5")

test_loss, test_acc = model.evaluate(test_dataset)
print('Test accuracy:', test_acc)

In [ ]:
# Transformer model
from tensorflow.keras import layers, models, Model
from tensorflow.keras.layers import Dense, LayerNormalization, Dropout, Input, MultiHeadAttention, Embedding

def create_transformer_model(input_shape, num_classes, num_heads=4, d_model=64, ff_dim=128, dropout_rate=0.1):
    # Input
    inputs = Input(shape=input_shape)

    # Embedding Layer
    x = layers.Dense(d_model)(inputs)  # Project input to d_model dimensions
    x = LayerNormalization(epsilon=1e-6)(x)

    # Multi-Head Attention and Feed-Forward
    for _ in range(2):  # Add 2 Transformer encoder layers
        attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(x, x)
        attn_output = Dropout(dropout_rate)(attn_output)
        x = layers.add([x, attn_output])  # Residual connection
        x = LayerNormalization(epsilon=1e-6)(x)

        ffn_output = Dense(ff_dim, activation="relu")(x)
        ffn_output = Dense(d_model)(ffn_output)
        ffn_output = Dropout(dropout_rate)(ffn_output)
        x = layers.add([x, ffn_output])  # Residual connection
        x = LayerNormalization(epsilon=1e-6)(x)

    # Pooling and Output
    x = layers.GlobalAveragePooling1D()(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(num_classes, activation="softmax")(x)

    # Create Model
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Example usage:
input_shape = (6, 1)  # Input shape (timesteps, features)
num_classes = 5       # Number of classes
model = create_transformer_model(input_shape=input_shape, num_classes=num_classes)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_dataset, validation_data=val_dataset, epochs=8)

# Save the model
# model.save("/kaggle/working/transformer_multi_app.h5")

# Evaluate the model
test_loss, test_acc = model.evaluate(test_dataset)
print("Test accuracy:", test_acc)


In [ ]:
# Hybrid model (LSTM, CNN and Transformer)
from tensorflow.keras import layers, models, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, LSTM, MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D

def create_hybrid_model(input_shape, num_classes):

    # Shared Input
    inputs = Input(shape=input_shape)

    # Branch 1: LSTM
    lstm_branch = LSTM(64, return_sequences=True)(inputs)
    lstm_branch = LSTM(64)(lstm_branch)

    # Branch 2: CNN
    cnn_branch = Conv1D(64, kernel_size=3, activation="relu", padding="same")(inputs)
    cnn_branch = MaxPooling1D(pool_size=2)(cnn_branch)
    cnn_branch = Flatten()(cnn_branch)

    # Branch 3: Transformer
    transformer_branch = Dense(64)(inputs)  # Project to Transformer dimensions
    transformer_branch = LayerNormalization(epsilon=1e-6)(transformer_branch)
    attn_output = MultiHeadAttention(num_heads=4, key_dim=64)(transformer_branch, transformer_branch)
    attn_output = Dropout(0.1)(attn_output)
    transformer_branch = layers.add([transformer_branch, attn_output])  # Residual connection
    transformer_branch = LayerNormalization(epsilon=1e-6)(transformer_branch)
    transformer_branch = GlobalAveragePooling1D()(transformer_branch)

    # Concatenate Branches
    concatenated = layers.concatenate([lstm_branch, cnn_branch, transformer_branch])

    # Fully Connected Layers
    x = Dense(128, activation="relu")(concatenated)
    x = Dropout(0.3)(x)

    # Output Layers
    classification_output = Dense(num_classes, activation="softmax", name="classification_output")(x)
    regression_output = Dense(1, name="regression_output")(x)

    # Model
    model = Model(inputs=inputs, outputs=[classification_output, regression_output])
    return model

# Example usage:
input_shape = (6, 1)  # Input shape (timesteps, features)
num_classes = 5       # Number of classes
model = create_hybrid_model(input_shape=input_shape, num_classes=num_classes)

# Compile the model
model.compile(optimizer='adam',
              loss={'classification_output': 'categorical_crossentropy',
                    'regression_output': 'mse'},
              metrics={'classification_output': 'accuracy',
                       'regression_output': 'mse'})

# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)

# Evaluate the model
test_results = model.evaluate(test_dataset)
print("Test Results:", test_results)


### Result:
Based on discription of CNN, LSTM and Transformer and amount of data we have is that we can not go with these models. Even these models are perfect and they can goes will on our data but because of lack of data we can not apply them using Neural Network. as my dataset is well for Trees structure we will try RandomForest and XGBoost to see. Just for classification, for regression we are going to Neural Network LSTM.

In [ ]:
x_train, x_val, x_test, y_train, y_val, y_test = split_data(new_data)

In [ ]:
# import the model
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state=42)

# train the model
rf_model.fit(x_train, y_train)

# Score on validation
val_score = rf_model.score(x_val, y_val)

# predict on test data
rf_pred = rf_model.predict(x_test)

# print the result
print(f"RandomForestClassifier on Validation data Score Result: {val_score}")

# print prediction on test dataset
print(f"RandomForestClassifier on Test data Prediction Result: {rf_pred}")

### Wow such a great Result 😍

In [ ]:
from sklearn.metrics import precision_score, roc_auc_score, accuracy_score, recall_score

# Evaluation function
def rf_eval(model, y_true, y_pred, average='weighted'):
    # Evaluate metrics
    roc = roc_auc_score(y_true, y_pred, average=average)
    preci = precision_score(y_true, y_pred, average=average)
    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred, average=average)

    # Print results
    print(f"Precision Score Result On Test data: {preci}")
    print(f"Roc_AUC Score Result On Test data: {roc}")
    print(f"Accuracy Score Result On Test data: {accuracy}")
    print(f"Recall Score Result On Test data: {recall}")

    return preci, roc, accuracy, recall

# Evaluate the model
preci, roc, accuracy, recall = rf_eval(rf_model, y_test, rf_pred)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

def rf_viz(model, y_true, y_pred, preci, roc, accuracy, recall, class_names=None):
    if len(y_true.shape) > 1 and y_true.shape[1] > 1:
        y_true = np.argmax(y_true, axis=1)
    if len(y_pred.shape) > 1 and y_pred.shape[1] > 1:
        y_pred = np.argmax(y_pred, axis=1)

    # Create a figure with subplots
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    # Confusion Matrix Visualization
    conf_matrix = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(conf_matrix, display_labels=class_names)
    disp.plot(cmap=plt.cm.Blues, ax=axes[0], colorbar=False)
    axes[0].set_title("Confusion Matrix", pad=15)  # Add padding to title

    # Visualize metrics as a bar chart
    metrics = {
        "Precision": preci,
        "Recall": recall,
        "Accuracy": accuracy,
        "ROC-AUC": roc
    }
    bars = axes[1].bar(metrics.keys(), metrics.values(), color=['blue', 'green', 'orange', 'red'])
    axes[1].set_ylabel("Score")
    axes[1].set_ylim(0, 1)
    axes[1].set_title("Model Evaluation Metrics", pad=20)  # Add padding to title

    # Add values on top of each bar
    for bar in bars:
        height = bar.get_height()
        axes[1].text(bar.get_x() + bar.get_width() / 2, height + 0.02, f"{height:.2f}", ha='center')

    plt.tight_layout()
    plt.show()

# Example usage
class_names = ['Good', 'Fair', 'Moderate', 'Abnormal', 'Dangerous']  # Replace with your actual class names
rf_viz(rf_model, y_test, rf_pred, preci, roc, accuracy, recall, class_names=class_names)


In [ ]:
from sklearn.model_selection import cross_val_score, KFold

k = 10
kfold = KFold(n_splits=k, shuffle=True, random_state=42)

# Perform k-fold cross-validation
cv_scores = cross_val_score(rf_model, x_train, y_train, cv=kfold, scoring='accuracy')

# Display cross-validation results
print(f"K-Fold Cross-Validation Scores: {cv_scores}")
print(f"Mean Accuracy: {np.mean(cv_scores):.4f}")
print(f"Standard Deviation: {np.std(cv_scores):.4f}")

# Optional: Check on test set to compare
test_score = rf_model.score(x_test, y_test)
print(f"RandomForestClassifier Test Score: {test_score:.4f}")

### Result:
All Right We have done it.<br>Time to celebrate.🎉🎉🎉🎉🎉🎉🎉😎
<br>Based on Discription of Evaluation Metrics. Our model learned well and need to be saved for real test.

In [ ]:
# Lets save the model
# import joblib
# joblib.dump(rf_model, "/content/drive/MyDrive/AFG-FTL-Capstone-Project/Models/rf_classification_model.joblib")

### Load RF Model:
We need to check feature importance Explainable AI and Decision Boundry to make sure model learned well.

In [ ]:
import joblib
rf_model = joblib.load("/content/drive/MyDrive/AFG-FTL-Capstone-Project/Models/rf_classification_model.joblib")
rf_model

In [ ]:
# RandomForest Feature Importance
importances = rf_model.feature_importances_

# features Our model trained on
features = ['co', 'no2', 'o3', 'so2', 'pm2_5', 'pm10']

# Sort the feature importances
indices = np.argsort(importances)[::-1]

# Plot
plt.figure(figsize=(10, 6))
plt.title("Feature Importance - Random Forest", fontsize=16)
plt.bar(range(len(features)), importances[indices], align="center")
plt.xticks(range(len(features)), [features[i] for i in indices], rotation=45, fontsize=12)
plt.ylabel("Importance Score", fontsize=14)
plt.xlabel("Features", fontsize=14)
plt.tight_layout()
plt.show()

### XGBoost Model:
As we have mentioned Earlier we are going to check XGBoost model also.

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, accuracy_score

if len(y_train.shape) > 1:
    y_train = np.argmax(y_train, axis=1)
    y_test = np.argmax(y_test, axis=1)
    y_val = np.argmax(y_val, axis=1)

# Define and train the XGBoost classifier
xgb_model = XGBClassifier(
    objective='multi:softprob',  # Multi-class classification
    num_class=5,  # Number of target classes
    eval_metric='mlogloss',  # Evaluation metric
    use_label_encoder=False,  # Prevent label encoding warnings
    random_state=42
)

# Fit the model
xgb_model.fit(x_train, y_train)

# Validate
xgb_val = xgb_model.score(x_val, y_val)
print(f"XGBoost Model Score on Validation data: {xgb_val}")

# Predict on the test set
xgb_pred = xgb_model.predict(x_test)

# Evaluate the model
accuracy = accuracy_score(y_test, xgb_pred)
print(f"XGBoost Model Accuracy: {accuracy:.4f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, xgb_pred, target_names=['Good', 'Fair', 'Moderate', 'Abnormal', 'Dangerous']))

# Confusion Matrix
disp = ConfusionMatrixDisplay.from_predictions(y_test, xgb_pred, display_labels=['Good', 'Fair', 'Moderate', 'Abnormal', 'Dangerous'], cmap="Blues")
disp.figure_.suptitle("Confusion Matrix")
plt.show()

### Result:
As you have seen our model learned and predict well on the data.<br>
So there is no need for further analysis.<br>
Just we need to Train Regression Model Using LSTM and Evaluate it.

### Regression Model training and checking up.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/AFG-FTL-Capstone-Project/processed_data.csv")
data

In [ ]:
data['aqi_shift_1h'] = data['aqi'].shift(-1)
data['aqi_shift_3h'] = data['aqi'].shift(-3)
data['aqi_shift_6h'] = data['aqi'].shift(-6)

In [ ]:
data.dropna(inplace=True)

In [ ]:
# Correct non-integer values in the specified columns
columns_to_fix = ['aqi_shift_48h', 'aqi_shift_72h']

for column in columns_to_fix:
    # Round values to the nearest integers and convert to float
    data[column] = data[column].round(0).astype(int)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
def split_data(dataset, target):
    """
    Splits the data into training, validation, and test sets (80% training, 5% validation, 15% test).
    """
    # first shuffle the data and then split

    dataset = shuffle(dataset, random_state=42)
    # Select features and target
    X = dataset.drop(['aqi_shift_24h', 'aqi_shift_48h', 'aqi_shift_72h','aqi_shift_1h','aqi_shift_3h','aqi_shift_6h', target], axis=1)

    # For future predictions, use the shifted columns as the target
    y = dataset[['aqi_shift_24h', 'aqi_shift_48h', 'aqi_shift_72h','aqi_shift_1h','aqi_shift_3h','aqi_shift_6h']]

    # Split data into training and remaining data (80% training, 20% remaining)
    X_train, X_remaining, y_train, y_remaining = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Split the remaining data into validation (5%) and test (15%) from the remaining 20%
    X_val, X_test, y_val, y_test = train_test_split(X_remaining, y_remaining, test_size=0.75, shuffle=False)

    return X_train, X_val, X_test, y_train, y_val, y_test


In [ ]:
import tensorflow as tf
def dataframe_to_tensor_regression(data):
    '''
    Convert Pandas DataFrame into TensorFlow Tensor for regression tasks,
    reshape features for LSTM/CNN models, and prepare tf.data.Dataset.
    '''
    # Set random seed
    tf.random.set_seed(1)

    # Train, validate, and test split
    x_train, x_val, x_test, y_train, y_val, y_test = split_data(data, 'aqi')

    # Convert DataFrame to TensorFlow Tensors
    x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
    x_val = tf.convert_to_tensor(x_val, dtype=tf.float32)
    x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)
    y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)  # Float for regression
    y_val = tf.convert_to_tensor(y_val, dtype=tf.float32)
    y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

    # Reshape features from 2D to 3D
    x_train = tf.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_val = tf.reshape(x_val, (x_val.shape[0], x_val.shape[1], 1))
    x_test = tf.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    # Convert to tf.data.Dataset
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

    # Shuffle and batch the datasets
    batch_size = 32
    train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

    return train_dataset, val_dataset, test_dataset, x_test, y_test

train_dataset, val_dataset, test_dataset, x_test, y_test = dataframe_to_tensor_regression(data)

In [ ]:
x_test.shape, y_test.shape

In [ ]:
time_steps = 10

def reshape_targets(y, time_steps):
    num_samples = y.shape[0] // time_steps  # Ensure it divides evenly
    y = y[:num_samples * time_steps]  # Trim extra rows that don't fit
    y_reshaped = y.reshape(num_samples, time_steps, -1)  # Reshape to 3D
    return y_reshaped

y_train_reshaped = reshape_targets(y_train, time_steps)
y_val_reshaped = reshape_targets(y_val, time_steps)
y_test_reshaped = reshape_targets(y_test, time_steps)

print("y_train_reshaped shape:", y_train_reshaped.shape)
print("y_val_reshaped shape:", y_val_reshaped.shape)
print("y_test_reshaped shape:", y_test_reshaped.shape)


In [ ]:
def reshape_for_lstm(X, time_steps):
    num_samples = X.shape[0] // time_steps  # Ensure we can divide into sequences
    X = X[:num_samples * time_steps]  # Trim extra rows that don't fit
    X_reshaped = X.reshape(num_samples, time_steps, X.shape[1])  # Reshape to 3D
    return X_reshaped

# Reshape your data
X_train_reshaped = reshape_for_lstm(X_train, time_steps)
X_val_reshaped = reshape_for_lstm(X_val, time_steps)
X_test_reshaped = reshape_for_lstm(X_test, time_steps)

# Print new shapes
print("X_train_reshaped shape:", X_train_reshaped.shape)
print("X_val_reshaped shape:", X_val_reshaped.shape)
print("X_test_reshaped shape:", X_test_reshaped.shape)


In [ ]:
y_test.shape

In [ ]:
y_train.shape

In [ ]:
import tensorflow as tf

# Define the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(75, 1)),  # Adjust input shape to match reshaped data
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(6)  # Output for 1h, 3h, 6h, 24h, 48h, 72h
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model using TensorFlow datasets
history = model.fit(
    train_dataset,  # Training data
    validation_data=val_dataset,  # Validation data
    epochs=20,
    verbose=1
)

# Evaluate the model
loss, mae = model.evaluate(test_dataset)
print(f"Test Loss: {loss}, Test MAE: {mae}")

# Predict for the test dataset
predictions = model.predict(test_dataset)

# If you need predictions for x_test (numpy array or tensor)
x_test_reshaped = tf.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))  # Reshape to 3D for LSTM
predictions_array = model.predict(x_test_reshaped)

print("Predictions shape:", predictions_array.shape)

In [ ]:
# model.save("/content/drive/MyDrive/AFG-FTL-Capstone-Project/Models/LSTM_regression.h5")

In [ ]:
from tensorflow.keras import losses, metrics

# Load the saved model with explicit loss and metrics
loaded_model = tf.keras.models.load_model(
    "/content/drive/MyDrive/AFG-FTL-Capstone-Project/Models/LSTM_regression.h5",
    custom_objects={"mse": losses.MeanSquaredError(), "mae": metrics.MeanAbsoluteError()}
)

# Evaluate the loaded model (optional)
loss, mae = loaded_model.evaluate(test_dataset)
print(f"Loaded Model Test Loss: {loss}, Test MAE: {mae}")

# Predict using the loaded model
x_test_reshaped = tf.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))  # Reshape to 3D for LSTM
predictions_array = loaded_model.predict(x_test_reshaped)
print(predictions_array)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LayerNormalization

# Define the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(75,1)),
    LayerNormalization(),
    tf.keras.layers.LSTM(32),
    LayerNormalization(),
    tf.keras.layers.Dense(6)
])


# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model using TensorFlow datasets
history = model.fit(
    train_dataset,  # Training data
    validation_data=val_dataset,  # Validation data
    epochs=10,
    verbose=1
)

# Evaluate the model
loss, mae = model.evaluate(test_dataset)
print(f"Test Loss: {loss}, Test MAE: {mae}")

# Predict for the test dataset
predictions = model.predict(test_dataset)

# If you need predictions for x_test (numpy array or tensor)
x_test_reshaped = tf.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))  # Reshape to 3D for LSTM
predictions_array = model.predict(x_test_reshaped)

print("Predictions shape:", predictions_array.shape)

In [ ]:
# model.save("/content/drive/MyDrive/AFG-FTL-Capstone-Project/Models/second_LSTM_regression.keras")

In [ ]:
# !pip install keras-tuner --upgrade

In [ ]:
import keras_tuner as kt
from tensorflow.keras.layers import LayerNormalization

def build_model(hp):
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(
            units=hp.Int("units_lstm1", min_value=32, max_value=128, step=32),  # Tune LSTM units
            return_sequences=True,
            input_shape=(75, 1)
        ),
        LayerNormalization(),
        tf.keras.layers.LSTM(
            units=hp.Int("units_lstm2", min_value=16, max_value=64, step=16),  # Tune LSTM units
        ),
        LayerNormalization(),
        tf.keras.layers.Dense(
            units=6
        )
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])  # Tune learning rate
        ),
        loss="mse",
        metrics=["mae"]
    )
    return model

tuner = kt.Hyperband(
    build_model,
    objective="val_mae",  # Optimize for validation MAE
    max_epochs=20,
    factor=3,
    directory="my_tuning_dir",
    project_name="lstm_tuning"
)

# Early stopping to prevent overfitting
stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

tuner.search(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,
    callbacks=[stop_early]
)


In [ ]:
# Retrieve the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperparameters:")
print(f"LSTM1 Units: {best_hps.get('units_lstm1')}")
print(f"LSTM2 Units: {best_hps.get('units_lstm2')}")
print(f"Learning Rate: {best_hps.get('learning_rate')}")

# Build the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Save the best model
best_model.save("/content/drive/MyDrive/AFG-FTL-Capstone-Project/Models/best_lstm_model.keras")
print("Best model saved as 'best_lstm_model.h5'")

### TSMixer Model Training

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_tsmixer_model(input_shape, output_units):
    """
    Builds a TSMixer model for regression.

    Parameters:
    - input_shape: Tuple representing the input shape (sequence_length, num_features).
    - output_units: Number of regression targets (output units).
    """
    # Input layer
    inputs = layers.Input(shape=input_shape)

    # Linear embedding of input features
    x = layers.Dense(128, activation='relu')(inputs)  # Feature embedding

    # First Mixer Layer: Time Mixing
    x = layers.Permute((2, 1))(x)  # Permute to mix across the time dimension
    x = layers.Dense(128, activation='relu')(x)  # Time mixing
    x = layers.Permute((2, 1))(x)  # Restore original dimensions
    x = layers.LayerNormalization()(x)

    # Second Mixer Layer: Feature Mixing
    x = layers.Dense(128, activation='relu')(x)  # Feature mixing
    x = layers.LayerNormalization()(x)

    # Global Average Pooling across the time dimension
    x = layers.GlobalAveragePooling1D()(x)

    # Output layer for regression
    outputs = layers.Dense(output_units, activation='linear')(x)  # Linear activation for regression

    # Build and compile the model
    model = models.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Adjust learning rate if needed
        loss='mse',  # Mean Squared Error for regression
        metrics=['mae']  # Mean Absolute Error as a metric
    )
    return model

# Define model parameters
sequence_length = 75  # Length of each sequence
num_features = 1      # Number of input features per timestep
output_units = 6      # Number of regression targets

# Build the TSMixer model
model = build_tsmixer_model((sequence_length, num_features), output_units)

# Summary of the model
model.summary()

# Train the model
history = model.fit(
    train_dataset,  # Training dataset (e.g., tf.data.Dataset)
    validation_data=val_dataset,  # Validation dataset
    epochs=10,  # Number of epochs
    batch_size=64,  # Batch size
    verbose=1
)

# Evaluate the model
loss, mae = model.evaluate(test_dataset)
print(f"Test Loss: {loss}, Test MAE: {mae}")

# Predict on test data
predictions = model.predict(x_test)
print("Predictions shape:", predictions.shape)

### TXMixer Hyper-prameter Tunning

In [ ]:
# Hyperprameter tunning of TSMixer
import keras_tuner as kt
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

# Define a function to build the TSMixer model
def build_tsmixer_model(hp):
    """
    Build a TSMixer model with tunable hyperparameters.

    Parameters:
    - hp: Hyperparameter tuner object.

    Returns:
    - Compiled TSMixer model.
    """
    inputs = layers.Input(shape=(75, 1))  # Input shape (sequence_length=75, num_features=1)

    # Linear embedding of input features
    x = layers.Dense(hp.Int("embedding_units", min_value=64, max_value=256, step=64), activation='relu')(inputs)

    # Add variable number of TSMixer layers
    for i in range(hp.Int("num_mixer_layers", min_value=1, max_value=4, step=1)):  # Vary the number of mixer layers
        # Time mixing
        x = layers.Permute((2, 1))(x)
        x = layers.Dense(hp.Int(f"units_time_mixer_{i+1}", min_value=64, max_value=256, step=64), activation='relu')(x)
        x = layers.Permute((2, 1))(x)
        x = layers.LayerNormalization()(x)

        # Feature mixing
        x = layers.Dense(hp.Int(f"units_feature_mixer_{i+1}", min_value=64, max_value=256, step=64), activation='relu')(x)
        x = layers.LayerNormalization()(x)

    # Global Average Pooling
    x = layers.GlobalAveragePooling1D()(x)

    # Output layer for regression
    outputs = layers.Dense(6, activation='linear')(x)  # Regression output (6 targets)

    # Build the model
    model = models.Model(inputs, outputs)

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])  # Tune learning rate
        ),
        loss='mse',
        metrics=['mae']
    )

    return model

# Initialize the tuner
tuner = kt.Hyperband(
    build_tsmixer_model,
    objective="val_mae",  # Minimize validation MAE
    max_epochs=50,
    factor=3,  # Reduction factor for the number of configurations
    directory="tsmixer_tuning",
    project_name="regression_tsmixer"
)

# Early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Start the hyperparameter search
tuner.search(
    train_dataset,  # Training dataset (tf.data.Dataset or numpy arrays)
    validation_data=val_dataset,  # Validation dataset
    epochs=50,
    callbacks=[early_stopping],
    verbose=1
)

# Get the best hyperparameters and model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print the best hyperparameters
print(f"""
The optimal number of embedding units is {best_hps.get('embedding_units')}.
The optimal number of mixer layers is {best_hps.get('num_mixer_layers')}.
The optimal learning rate is {best_hps.get('learning_rate')}.
""")

# Build the best model
best_model = tuner.hypermodel.build(best_hps)

# Train the best model with early stopping
history = best_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    callbacks=[early_stopping],
    verbose=1
)

# Save the model
best_model.save("/content/drive/MyDrive/AFG-FTL-Capstone-Project/Models/best_tsmixer_model.keras")

# Evaluate the model on the test dataset
loss, mae = best_model.evaluate(test_dataset)
print(f"Test Loss: {loss}, Test MAE: {mae}")

# Predictions
predictions = best_model.predict(x_test)
print("Predictions shape:", predictions.shape)


### Loading & Evaluating TSMixer Model:
After Hyper-prameter Tunning of TSMixer Model now it is time to evaluate it.

In [ ]:
from tensorflow.keras.models import load_model


# Load the saved TSMixer model
model_path = "/content/drive/MyDrive/AFG-FTL-Capstone-Project/Models/best_tsmixer_model.keras"
model = load_model(model_path)

# Summary of the model
model.summary()

In [ ]:
# Evaluate the model on the test dataset
loss, mae = model.evaluate(test_dataset, verbose=1)

# Print the evaluation metrics
print(f"Test Loss (MSE): {loss}")
print(f"Test Mean Absolute Error (MAE): {mae}")

### Evaluating the Model on Test data.
The model is evaluating on Mean Squared Error and R square error.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Predictions
y_pred = model.predict(x_test)
y_true = y_test

y_pred = y_pred.numpy() if isinstance(y_pred, tf.Tensor) else y_pred
y_true = y_true.numpy() if isinstance(y_true, tf.Tensor) else y_true

# Compute RMSE
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Compute R² Score
r2 = r2_score(y_true, y_pred)
print(f"R² Score: {r2}")

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error


# Convert train_dataset to numpy arrays
x_data = []
y_data = []

for x_batch, y_batch in train_dataset:
    x_data.append(x_batch.numpy())
    y_data.append(y_batch.numpy())

# Combine into full arrays
x_data = np.concatenate(x_data, axis=0)
y_data = np.concatenate(y_data, axis=0)

# K-fold cross validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)


# Store evaluation results for each fold
mse_scores = []
mae_scores = []
rmse_scores = []
r2_scores = []

fold = 1
for train_index, val_index in kf.split(x_data):
    print(f"\nFold {fold}")

    # Split data into train and validation sets
    x_train_fold, x_val_fold = x_data[train_index], x_data[val_index]
    y_train_fold, y_val_fold = y_data[train_index], y_data[val_index]

    # Reload the model for each fold
    model = load_model(model_path)

    # Train the model on the current fold
    history = model.fit(
        x_train_fold, y_train_fold,
        validation_data=(x_val_fold, y_val_fold),
        epochs=15,
        batch_size=64,
        verbose=1,
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)]
    )

    # Predict on validation fold
    y_pred_fold = model.predict(x_val_fold)

    # Calculate metrics for the current fold
    mse = mean_squared_error(y_val_fold, y_pred_fold)
    mae = mean_absolute_error(y_val_fold, y_pred_fold)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_val_fold, y_pred_fold)

    print(f"Fold {fold} - MSE: {mse}, MAE: {mae}, RMSE: {rmse}, R²: {r2}")

    # Append metrics to the results
    mse_scores.append(mse)
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    r2_scores.append(r2)

    fold += 1

# Print overall metrics across all folds
print("\nCross-Validation Results:")
print(f"Average MSE: {np.mean(mse_scores)}")
print(f"Average MAE: {np.mean(mae_scores)}")
print(f"Average RMSE: {np.mean(rmse_scores)}")
print(f"Average R²: {np.mean(r2_scores)}")

In [ ]:
import matplotlib.pyplot as plt

# Example: y_true and y_pred after cross-validation
y_true = y_val_fold.flatten()
y_pred = model.predict(x_val_fold).flatten()

plt.figure(figsize=(8, 8))
plt.scatter(y_true, y_pred, alpha=0.5, label="Predicted vs Actual")
plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', label="Ideal Fit")
plt.xlabel("True Values")
plt.ylabel("Predicted Values")
plt.legend()
plt.title("True vs Predicted Values")
plt.show()

In [ ]:
residuals = y_true - y_pred

plt.figure(figsize=(8, 6))
plt.scatter(y_pred, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.title("Residual Plot")
plt.show()

### Explainable AI:
Using SHAP for checking the output result.

In [ ]:
import shap

# Extract data from train_dataset
x_data = []
for x_batch, _ in train_dataset:
    x_data.append(x_batch.numpy())  # Convert each batch to NumPy

# Concatenate all batches into a single NumPy array
x_data = np.concatenate(x_data, axis=0)  # Combine batches along the sample axis
x_data = x_data.reshape(-1, 75, 1)  # Reshape to match model input shape

# Define the model prediction wrapper
def model_predict(inputs):
    # Ensure inputs are converted to the correct shape for the model
    inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)  # Convert to Tensor
    return model(inputs).numpy()  # Run model and convert predictions to NumPy

explainer = shap.KernelExplainer(model_predict, x_data)  # Use a subset as baseline
shap_values = explainer.shap_values(x_data)
shap.summary_plot(shap_values, x_data)

In [ ]:
import shap
import tensorflow as tf
import numpy as np

# Enable NumPy-like behavior in TensorFlow
tf.experimental.numpy.experimental_enable_numpy_behavior()

# Load your trained TensorFlow model
model = tf.keras.models.load_model("/content/drive/MyDrive/AFG-FTL-Capstone-Project/Models/best_tsmixer_model.keras")  # Replace with actual path

# Prepare a subset of the test data for SHAP
x_test_sample = x_test[:100]  # Use a smaller subset for SHAP (e.g., 100 samples)

# Convert the test data to NumPy arrays
x_test_sample_np = x_test_sample.numpy()  # Convert to NumPy
y_test_sample_np = y_test[:100].numpy()   # Optional: Convert y_test if needed

# Define a function for predictions
# Define a prediction function
def predict(input_data):
    return model.predict(input_data)

# Flatten the input data
x_test_sample_flat = x_test_sample_np.reshape(x_test_sample_np.shape[0], -1)  # Flatten timesteps and features

# Initialize SHAP KernelExplainer with flattened data
explainer = shap.KernelExplainer(predict, x_test_sample_flat)

# Calculate SHAP values
shap_values = explainer.shap_values(x_test_sample_flat)

# Visualize SHAP values
shap.summary_plot(shap_values, x_test_sample_flat)

# Visualize single-instance explanations
shap.force_plot(explainer.expected_value[0], shap_values[0][0], x_test_sample_flat[0])

### It is time to Train our Reinforcment Learning Model:
In Reinforcment learning we are going to use PPO model, because it is a policy optimization and we just need that based on our policy make a decision.
<br>
**Steps we follow:**<br>
* Creating Envirnoment
* Train PPO model

In [ ]:
import gym
from gym import spaces
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# Define the custom environment
class AirQualityEnv(gym.Env):
    def __init__(self, regression_predictions, actual_values, external_features=None):
        """
        Custom environment for PPO with air quality predictions.

        :param regression_predictions: np.array, predictions from the regression model
        :param actual_values: np.array, ground truth values for reward calculation
        :param external_features: np.array, external features (optional, can be None)
        """
        super(AirQualityEnv, self).__init__()

        # Initialize predictions, actual values, and external features
        self.regression_predictions = regression_predictions
        self.actual_values = actual_values
        self.external_features = external_features if external_features is not None else np.zeros((5,))

        # Define the action space (adjustments to predictions)
        self.action_space = spaces.Box(low=-1, high=1, shape=(len(self.regression_predictions),), dtype=np.float32)

        # Define the observation space (predictions + external features)
        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(len(self.regression_predictions) + len(self.external_features),),
            dtype=np.float32
        )

        # Combine predictions and external features into the state
        self.state = np.concatenate([self.regression_predictions, self.external_features])

    def step(self, action):
        """
        Execute one step in the environment.

        :param action: np.array, adjustments to predictions
        :return: tuple (state, reward, done, info)
        """
        # Apply action (adjust predictions)
        adjusted_predictions = self.regression_predictions + action

        # Calculate reward (negative mean absolute error as an example)
        reward = -np.mean(np.abs(adjusted_predictions - self.actual_values))

        # Update state
        self.state = np.concatenate([adjusted_predictions, self.external_features])

        # End episode after one step (for simplicity)
        done = True

        # Additional info (empty for now)
        info = {}

        return self.state, reward, done, info

    def reset(self):
        """
        Reset the environment state.
        """
        self.state = np.concatenate([self.regression_predictions, self.external_features])
        return self.state

# Example data
regression_predictions = np.array([1, 2, 3, 4, 5])  # Replace with actual predictions
actual_values = np.array([2, 3, 4, 5, 6])           # Replace with ground truth
external_features = np.array([0.1, 0.2, 0.3, 0.4, 0.5])  # Replace with external features if available

# Instantiate the environment
env = AirQualityEnv(regression_predictions, actual_values, external_features)

# Vectorize the environment (for stability and parallel training)
vec_env = make_vec_env(lambda: env, n_envs=1)

# Create the PPO model
ppo_model = PPO("MlpPolicy", vec_env, verbose=1)

# Train the PPO model
ppo_model.learn(total_timesteps=10000)

# Save the model
ppo_model.save("ppo_air_quality_model.zip")

# Load the model (if needed later)
ppo_model = PPO.load("ppo_air_quality_model.zip")

# Test the model
obs = env.reset()
action, _ = ppo_model.predict(obs)
adjusted_predictions = regression_predictions + action

print("Adjusted Predictions:", adjusted_predictions)
